In [3]:
import pandas as pd
import numpy as np


In [8]:
df2 = pd.read_csv("https://raw.githubusercontent.com/JackronyK/SLA_ETL/main/Processsed_Data/SAFARICOM_Invoice.csv")
df = pd.read_csv("https://raw.githubusercontent.com/JackronyK/SLA_ETL/main/Processsed_Data/LTK_Invoice.csv")

In [9]:
df.head()

,Invoice Date,Invoice No,Invoice Period,Link ID,Invoice Description,QRC (EX TAX),Excise (15%),VAT (16%),Invoice Total
0,2023-10-13,CRN-15473,01-Oct-2023 to 31-Dec-2023,26046,26046 National MPLS KRA MBITA TO VRF (INS-57),-48264.0,0,-7722.24,-55986.24
1,2023-10-06,116426,01-Oct-2023 to 31-Dec-2023,C-00159-0087,C-00159-0087 EPL Connection charge 80MBPS->SAM...,360000.0,0,57600.00,417600.00
2,2023-10-06,116426,01-Oct-2023 to 31-Dec-2023,1750,01750 National MPLS KRA - TIMES TOWERS TO VRF,810000.0,0,129600.00,939600.00
3,2023-10-06,116426,01-Oct-2023 to 31-Dec-2023,5611,05611 National MPLS Msa link vlan 460 customs,61927.5,0,9908.40,71835.90
4,2023-10-06,116426,01-Oct-2023 to 31-Dec-2023,20103,20103 Leased_Circuits_Naticnal KRA WILSON AIRP...,68850.0,0,11016.00,79866.00


In [8]:
df2.tail()

,No.,Invoice Date,Invoice Ref.,Invoice Period,Link ID,Invoice Description,Net billed(Exc. Taxes),VAT,Excise,Monthly Total Billed,Period(Months),Total Billed for the Period,Amount Including Excise
54,55,2023-10-01,B1-10096515502,1st Oct to 31st Dec 2023,95096538.0,KRA BUSIA-WIMAX,7.000830e+04,12881.527200,10501.24500,9.339107e+04,3.0,280173.2166,8.050954e+04
55,56,2023-10-01,B1-10096515502,1st Oct to 31st Dec 2023,95096687.0,KRA CUSTOMS OFFICE- NAIROBI-NAMANGA WIMAX,6.999750e+04,12879.540000,10499.62500,9.337666e+04,3.0,280129.9950,8.049712e+04
56,57,2023-10-01,B1-10096515502,1st Oct to 31st Dec 2023,95096716.0,KRA ISEBANIA-WIMAX,7.368510e+04,13558.058400,11052.76500,9.829592e+04,3.0,294887.7702,8.473787e+04
57,58,2023-10-01,B1-10096515502,1st Oct to 31st Dec 2023,95096873.0,KRA LUNGA LUNGA WIMAX,6.999750e+04,12879.540000,10499.62500,9.337666e+04,3.0,280129.9950,8.049712e+04
58,Total,NaN,NaN,NaN,NaN,NaN,2.815170e+06,517991.341566,422275.55019,3.755437e+06,NaN,NaN,3.237446e+06


### Codeing before transforming them into classes

In [12]:
# Dropping rows with null values in all the fields
df2.dropna(how='all', inplace = True)

#dropping the total row
df2.drop(df2[df2["No."] == "Total"].index, inplace= True)

##Function to help pick the columns
def pick_Invoice_no_column_name(df):
    # Loop through the fixed substrings
    for substring in ['inveoice no', 'ref']:
        # Check if any column contains the substring
        matching_columns = [col for col in df.columns if substring.lower() in col.lower()]
        # If a matching column is found, return its name
        if matching_columns:
            return matching_columns[0]
        
def QRC_Finder(df):
    max_value = None
    max_col = None
    for substring in ['QRC', 'Total', 'Amount']:
        #Checking for the matching amount related columns
        fields = [col for col in df.columns if substring.lower() in col.lower()]
    for field in fields:
        # Check if the column exists and has valid values
        if field in df.columns and not df[field].isnull().all():
            # Finding the maximum value in the first row of the column
            first_row_value = df.at[1,field]
            if max_value is None or first_row_value > max_value:
                max_value = first_row_value
                max_col = field
    return max_col



# Dropping unwanted Columns
columns_wanted = {"Date": "Invoice Date",
                  "ID": "Link ID",
                   "Period": "Invoice Period",
                    "Descriprion": "Invoice Description",
                     "QRC":QRC_Finder(df2),
                      "Reference/ no": pick_Invoice_no_column_name(df2)}

df_selected = df2.loc[:, [value for value in columns_wanted.values()]]



In [13]:
df_selected.head()

,Invoice Date,Link ID,Invoice Period,Invoice Description,Amount Including Excise,Invoice Ref.
0,2023-10-01,95001135.0,1st Oct to 31st Dec 2023,CUSTOMS OFFICE MALINDI-WIMAX,69007.36,B1-10096515502
1,2023-10-01,95001574.0,1st Oct to 31st Dec 2023,KRA LOITOKTOK BORDER WIMAX,69007.36,B1-10096515502
2,2023-10-01,95001628.0,1st Oct to 31st Dec 2023,KRA KRATI MOMBASA WIMAX,69007.36,B1-10096515502
3,2023-10-01,95004139.0,1st Oct to 31st Dec 2023,KRA - MANDERA,51746.78,B1-10096515502
4,2023-10-01,95004140.0,1st Oct to 31st Dec 2023,KRA - Moyale,69007.36,B1-10096515502


#### function Workstation

In [29]:
columns_wanted = {"Date": "Invoice Date",
                  "ID": "Link ID",
                   "Period": "Invoice Period",
                    "Descriprion": "Invoice Description",
                     "QRC":"QRC_Finder(df2)",
                      "Reference/ no": "pick_Invoice_no_column_name(df2)"}
list(columns_wanted.values())

['Invoice Date',
 'Link ID',
 'Invoice Period',
 'Invoice Description',
 'QRC_Finder(df2)',
 'pick_Invoice_no_column_name(df2)']

In [24]:
not df2['Amount Including Excise'].isnull().all()


True

In [25]:
df2.at[0,'Invoice Date']

'2023-10-01'

In [21]:
def QRC_Finder(df):
    max_value = None
    max_column = None
    
    for substring in ['QRC', 'Total', 'Amount']:
        # Checking for the matching amount related columns
        fields = [col for col in df.columns if substring.lower() in col.lower()]
    
    for field in fields:
        # Check if the column exists and has valid values
        if field in df.columns and not df[field].isnull().all():
            # Find the maximum value in the first row of the column
            first_row_value = df.at[0, field]
            if max_value is None or first_row_value > max_value:
                max_value = first_row_value
                max_column = field
    
    return max_column

# Example usage
df = pd.DataFrame({'Total Amount': [100, 150, 200], 'Invoice Amount': [120, 140, 180]})
largest_column = QRC_Finder(df)
print("Column with largest value:", largest_column)


Column with largest value: Invoice Amount


In [ ]:
df2.tail()

In [18]:
import pandas as pd

def pick_column(df, substrings):
    # Initialize an empty list to store the filtered columns
    wanted_columns = []
    
    # Loop through each substring and filter columns
    for substring in substrings:
        # Filter columns containing the current substring
        columns = df.columns[df.columns.str.contains(substring, case=False)]
        # Extend the list of wanted columns with the filtered columns
        wanted_columns.extend(columns)
    
    # Remove duplicates and return the final list of wanted columns
    return list(set(wanted_columns))

# Example usage
df = pd.DataFrame({'Invoice Ref': [1, 2, 3], 'Invoice Kenya.': [4, 5, 6]})
wanted_columns = pick_column(df, ['no.', 'ref'])
print(wanted_columns)


['Invoice Ref']


In [20]:
import pandas as pd

def pick_column_name(df):
    # Loop through the fixed substrings
    for substring in ['no.', 'ref']:
        # Check if any column contains the substring
        matching_columns = [col for col in df.columns if substring.lower() in col.lower()]
        # If a matching column is found, return its name
        if matching_columns:
            return matching_columns[0]

# Example usage
df = pd.DataFrame({'Invoice Ref': [1, 2, 3], 'Invoice kenya.': [4, 5, 6]})
wanted_column = pick_column_name(df)
print(wanted_column)


Invoice Ref


### Classes

### Invoice Cleaner Class

In [39]:
import pandas as pd

class InvoiceCleaner:
    def __init__(self, df):
        self.df = df
        
    def drop_null_rows(self):
        self.df.dropna(how='all', inplace=True)
        
    def drop_total_row(self):
        if "No." in self.df.columns:
            self.df.drop(self.df[self.df["No."] == "Total"].index, inplace=True)
        
        
    def pick_invoice_no_column_name(self):
        for substring in ['invoice no', 'ref']:
            matching_columns = [col for col in self.df.columns if substring.lower() in col.lower()]
            if matching_columns:
                return matching_columns[0]
                
    def QRC_Finder(self):
        max_value = None
        max_col = None
        for substring in ['QRC', 'Total', 'Amount']:
            fields = [col for col in self.df.columns if substring.lower() in col.lower()]
            for field in fields:
                if field in self.df.columns and not self.df[field].isnull().all():
                    first_row_value = self.df.at[1, field]
                    if max_value is None or first_row_value > max_value:
                        max_value = first_row_value
                        max_col = field
        return max_col
    
    def clean_invoice_data(self):
        self.drop_null_rows()
        self.drop_total_row()
        
        columns_wanted = {
            "Date": "Invoice Date",
            "ID": "Link ID",
            "Period": "Invoice Period",
            "Descriprion": "Invoice Description",
            "QRC": self.QRC_Finder(),
            "Reference/ no": self.pick_invoice_no_column_name()
        }

        self.df = self.df.loc[:, [value for value in columns_wanted.values()]]

    def adjust_col_dtypes(self):
        #Converting the Invoice date col into Date dtype
        if 'Invoice Date' in self.df.columns:
            self.df['Invoice Date'] = pd.to_datetime(self.df['Invoice Date'])
        
        #Adjusting the link ID based on current dtype
        if 'Link ID' in self.df.columns:
            if self.df['Link ID'].dtype == 'float':
                self.df['Link ID'] = self.df['Link ID'].astype('int').astype('str')
            elif self.df['Link ID'].dtype == 'int':
                self.df['Link ID'] = self.df['Link ID'].astype('str')


In [38]:
# Assuming df2 is your DataFrame
cleaner = InvoiceCleaner(df2)
cleaner.adjust_col_dtypes()
cleaned_df = cleaner.df
cleaned_df.head()


,No.,Invoice Date,Invoice Ref.,Invoice Period,Link ID,Invoice Description,Net billed(Exc. Taxes),VAT,Excise,Monthly Total Billed,Period(Months),Total Billed for the Period,Amount Including Excise
0,1,2023-10-01,B1-10096515502,1st Oct to 31st Dec 2023,95001135,CUSTOMS OFFICE MALINDI-WIMAX,60006.4,11041.1776,9000.96,80048.5376,3.0,240145.6128,69007.36
1,2,2023-10-01,B1-10096515502,1st Oct to 31st Dec 2023,95001574,KRA LOITOKTOK BORDER WIMAX,60006.4,11041.1776,9000.96,80048.5376,3.0,240145.6128,69007.36
2,3,2023-10-01,B1-10096515502,1st Oct to 31st Dec 2023,95001628,KRA KRATI MOMBASA WIMAX,60006.4,11041.1776,9000.96,80048.5376,3.0,240145.6128,69007.36
3,4,2023-10-01,B1-10096515502,1st Oct to 31st Dec 2023,95004139,KRA - MANDERA,44997.2,8279.4848,6749.58,60026.2648,3.0,180078.7944,51746.78
4,5,2023-10-01,B1-10096515502,1st Oct to 31st Dec 2023,95004140,KRA - Moyale,60006.4,11041.1776,9000.96,80048.5376,3.0,240145.6128,69007.36


In [63]:
# Assuming df2 is your DataFrame
cleaner = InvoiceCleaner(df)
cleaner.clean_invoice_data()
cleaned_df = cleaner.df
cleaned_df.head()

,Invoice Date,Link ID,Invoice Period,Invoice Description,Invoice Total,Invoice No
0,2023-10-13,26046,01-Oct-2023 to 31-Dec-2023,26046 National MPLS KRA MBITA TO VRF (INS-57),-55986.24,CRN-15473
1,2023-10-06,C-00159-0087,01-Oct-2023 to 31-Dec-2023,C-00159-0087 EPL Connection charge 80MBPS->SAM...,417600.00,116426
2,2023-10-06,1750,01-Oct-2023 to 31-Dec-2023,01750 National MPLS KRA - TIMES TOWERS TO VRF,939600.00,116426
3,2023-10-06,5611,01-Oct-2023 to 31-Dec-2023,05611 National MPLS Msa link vlan 460 customs,71835.90,116426
4,2023-10-06,20103,01-Oct-2023 to 31-Dec-2023,20103 Leased_Circuits_Naticnal KRA WILSON AIRP...,79866.00,116426


In [18]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Invoice Date                 58 non-null     object 
 1   Link ID                      58 non-null     float64
 2   Invoice Period               58 non-null     object 
 3   Invoice Description          58 non-null     object 
 4   Total Billed for the Period  58 non-null     float64
 5   Invoice Ref.                 58 non-null     object 
dtypes: float64(2), object(4)
memory usage: 2.8+ KB


#### Enhancements

In [34]:
cleaned_df['Invoice Date'] = pd.to_datetime(cleaned_df['Invoice Date'])
cleaned_df['Link ID'] = cleaned_df['Link ID'].astype('int').astype('str')
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Invoice Date                 58 non-null     datetime64[ns]
 1   Link ID                      58 non-null     object        
 2   Invoice Period               58 non-null     object        
 3   Invoice Description          58 non-null     object        
 4   Total Billed for the Period  58 non-null     float64       
 5   Invoice Ref.                 58 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 2.8+ KB


In [35]:
cleaned_df.head()

,Invoice Date,Link ID,Invoice Period,Invoice Description,Total Billed for the Period,Invoice Ref.
0,2023-10-01,95001135,1st Oct to 31st Dec 2023,CUSTOMS OFFICE MALINDI-WIMAX,240145.6128,B1-10096515502
1,2023-10-01,95001574,1st Oct to 31st Dec 2023,KRA LOITOKTOK BORDER WIMAX,240145.6128,B1-10096515502
2,2023-10-01,95001628,1st Oct to 31st Dec 2023,KRA KRATI MOMBASA WIMAX,240145.6128,B1-10096515502
3,2023-10-01,95004139,1st Oct to 31st Dec 2023,KRA - MANDERA,180078.7944,B1-10096515502
4,2023-10-01,95004140,1st Oct to 31st Dec 2023,KRA - Moyale,240145.6128,B1-10096515502


In [ ]:
cleaned_df['Link ID'].astype('object')

In [80]:
import pandas as pd

class DataCleaner:
    def __init__(self, df):
        self.df = df

    def drop_null_rows(self):
        self.df.dropna(how='all', inplace=True)

    def drop_total_row(self):
        if "No." in self.df.columns:
            self.df.drop(self.df[self.df["No."] == "Total"].index, inplace=True)

    def pick_invoice_no_column_name(self):
        for substring in ['invoice no', 'ref']:
            matching_columns = [col for col in self.df.columns if substring.lower() in col.lower()]
            if matching_columns:
                return matching_columns[0]

    def QRC_Finder(self):
        max_value = None
        max_col = None
        for substring in ['QRC', 'Total', 'Amount']:
            fields = [col for col in self.df.columns if substring.lower() in col.lower()]
            for field in fields:
                if field in self.df.columns and not self.df[field].isnull().all():
                    first_row_value = self.df.at[1, field]
                    if max_value is None or first_row_value > max_value:
                        max_value = first_row_value
                        max_col = field
        return max_col

    def col_of_interest(self):
        self.drop_null_rows()
        self.drop_total_row()

        columns_wanted = {
            "Date": "Invoice Date",
            "ID": "Link ID",
            "Period": "Invoice Period",
            "Description": "Invoice Description",            
            "Reference/No": self.pick_invoice_no_column_name(),
            "QRC": self.QRC_Finder()
        }

        self.df = self.df.loc[:, [value for value in columns_wanted.values()]]


    def col_renaming(self):
        self.col_of_interest()

        new_col_names = {
            "Invoice Date": "Invoice_Data",
            "Link ID": "Link_ID",
            "Invoice Period": "Invoice_Period",
            "Invoice Description":"Invoice_Description",           
            self.pick_invoice_no_column_name(): "Invoice_Reference",
             self.QRC_Finder(): "Total_QRC"
        }
        self.df = self.df.rename(columns = new_col_names)

    def clean_invoice_data(self):
        self.col_renaming()
        # Convert 'Invoice Date' to datetime
        if 'Invoice_Date' in self.df.columns:
            self.df['Invoice_Date'] = pd.to_datetime(self.df['Invoice_Date'])

        # Adjust 'Link ID' based on its current type
        if 'Link ID' in self.df.columns:
            if self.df['Link_ID'].dtype == 'float':
                self.df['Link_ID'] = self.df['Link_ID'].astype('int').astype('str')
            elif self.df['Link_ID'].dtype == 'int':
                self.df['Link_ID'] = self.df['Link_ID'].astype('str')
            # If 'Link ID' is already object, no conversion is needed
        return self.df

# Assuming df is your DataFrame
cleaner = InvoiceCleaner(df)
cleaned_df = cleaner.clean_invoice_data()



In [81]:
cleaned_df.head()

,Invoice_Data,Link_ID,Invoice_Period,Invoice_Description,Invoice_Reference,Total_QRC
0,2023-10-13,26046,01-Oct-2023 to 31-Dec-2023,26046 National MPLS KRA MBITA TO VRF (INS-57),CRN-15473,-55986.24
1,2023-10-06,C-00159-0087,01-Oct-2023 to 31-Dec-2023,C-00159-0087 EPL Connection charge 80MBPS->SAM...,116426,417600.00
2,2023-10-06,1750,01-Oct-2023 to 31-Dec-2023,01750 National MPLS KRA - TIMES TOWERS TO VRF,116426,939600.00
3,2023-10-06,5611,01-Oct-2023 to 31-Dec-2023,05611 National MPLS Msa link vlan 460 customs,116426,71835.90
4,2023-10-06,20103,01-Oct-2023 to 31-Dec-2023,20103 Leased_Circuits_Naticnal KRA WILSON AIRP...,116426,79866.00


In [82]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Invoice_Data         26 non-null     object 
 1   Link_ID              26 non-null     object 
 2   Invoice_Period       26 non-null     object 
 3   Invoice_Description  26 non-null     object 
 4   Invoice_Reference    26 non-null     object 
 5   Total_QRC            26 non-null     float64
dtypes: float64(1), object(5)
memory usage: 1.3+ KB


In [67]:
cleaned_df

,Invoice Date,Link ID,Invoice Period,Invoice Description,Invoice Total,Invoice No
0,2023-10-13,26046,01-Oct-2023 to 31-Dec-2023,26046 National MPLS KRA MBITA TO VRF (INS-57),-55986.240,CRN-15473
1,2023-10-06,C-00159-0087,01-Oct-2023 to 31-Dec-2023,C-00159-0087 EPL Connection charge 80MBPS->SAM...,417600.000,116426
2,2023-10-06,1750,01-Oct-2023 to 31-Dec-2023,01750 National MPLS KRA - TIMES TOWERS TO VRF,939600.000,116426
3,2023-10-06,5611,01-Oct-2023 to 31-Dec-2023,05611 National MPLS Msa link vlan 460 customs,71835.900,116426
4,2023-10-06,20103,01-Oct-2023 to 31-Dec-2023,20103 Leased_Circuits_Naticnal KRA WILSON AIRP...,79866.000,116426
5,2023-10-06,20954,01-Oct-2023 to 31-Dec-2023,20954 National MPLS KRA EGERTON UNIVERSITY NJO...,174000.000,116426
6,2023-10-06,21195,01-Oct-2023 to 31-Dec-2023,21195 National MPLS KRA SAMEER BUSINESS PARK T...,501120.000,116426
7,2023-10-06,23945,01-Oct-2023 to 31-Dec-2023,23945 National MPLS KRA JKIA TERMINAL 2 TO VRF...,69600.000,116426
8,2023-10-06,24174,01-Oct-2023 to 31-Dec-2023,24174 National MPLS KRA KISUMU PIER TO VRF(#48...,66120.000,116426
9,2023-10-06,25102,01-Oct-2023 to 31-Dec-2023,25102 National MPLS KRA KITALE TO VRF (#45509),187920.000,116426


In [ ]:
cleaned_df.rename({""})

## SLA Cleaner

In [1]:
import pandas as pd
df_saf = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\SLA_ETL\\Processsed_Data\\SAFARICOM_SLA.csv')
df_ltk = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\SLA_ETL\\Processsed_Data\\LTK_SLA.csv')



In [3]:
df_ltk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   No.               26 non-null     int64  
 1   SLA Service       26 non-null     object 
 2   Link Description  24 non-null     object 
 3   Link ID           26 non-null     object 
 4   Capacity (Mbps)   26 non-null     int64  
 5   MRC               26 non-null     float64
 6   QRC               26 non-null     float64
 7   Last Mile         26 non-null     object 
 8   SLA Date          23 non-null     object 
 9   Status            26 non-null     object 
 10  SLM Comments      26 non-null     object 
dtypes: float64(2), int64(2), object(7)
memory usage: 2.4+ KB


In [ ]:
df_saf.tail()

### The SLA Cleaner WS

In [81]:
class SLA_Cleaner:
    def __init__(self, df):
          self.df = df
         
    def drop_null_rows(self):
            self.df.dropna(how='all', inplace=True)

    def drop_total_row(self):
        if "No." in self.df.columns:
            self.df.drop(self.df[self.df["No."] == "Total"].index, inplace=True)
    def capacity_picker(self):
        for pat in ['Capacity']:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]
    def comments_picker(self):
        for pat in ['comment']:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]
            
    def Site_desc_picker(self):
        for pat in ['site', 'description']:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]
    def MRC_picker(self):
        for pat in ['month', 'MRC']:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]
            
    def SLA_col_subsetting(self):
        self.capacity_picker()
        self.comments_picker()
        self.Site_desc_picker()
        self.MRC_picker()
        
        SLA_Cols = {'Link ID': 'Link ID',
            'SLA Dates':'SLA Date',
            'Last Mile': 'Last Mile',            
            'Capacity': self.capacity_picker(),
            'Sites/Location': self.Site_desc_picker(),
            'MRC': self.MRC_picker(),
            'Comments':self.comments_picker()
            }
        self.df = self.df.loc[:, [col for col in SLA_Cols.values()]]

        # Transforming the SLA Cols
    def cols_transfomer(self):
        self.SLA_col_subsetting()
        self.excise_duty = 0.15
        self.VAT = 0.16
        # Changing the link id to str depending in the current dtype
        if 'Link ID' in self.df.columns:
            if self.df['Link ID'].dtype == 'float':
                self.df['Link ID'] = self.df['Link ID'].astype('int').astype('str')
            elif self.df['Link ID'].dtype == 'int':
                self.df['Link ID'] = self.df['Link ID'].astype('str')
        
        #Date to dtype Date
        if 'SLA Date' in self.df.columns:
            self.df['SLA Date'] = pd.to_datetime(self.df['SLA Date'])
        
        # Capacity
        if self.capacity_picker() in self.df.columns and self.df[self.capacity_picker()].dtype == 'object':
            self.df[self.capacity_picker()] = self.df[self.capacity_picker()].str.split(' ',expand =True)[0].astype('int')
        
        # Calculating the Quarterly Recurring Charges
        if self.MRC_picker() in self.df.columns:
            self.df['QRC'] = np.multiply(np.multiply(self.df[self.MRC_picker()],
                                                (1 + self.excise_duty + self.VAT)),3)
        
        #renaming the Columns
        SLA_New_Names = {
            'Link ID': 'Link_ID',
            'SLA Date': 'SLA_Date',
            'Last Mile': 'Last_Mile',
            'QRC': 'QRC_Incl',
            self.Site_desc_picker(): 'Location',
            self.capacity_picker(): 'Capacity',
            self.MRC_picker(): 'MRC_Excl',
            self.comments_picker(): 'SLM_Comments'
        }
        self.df = self.df.rename(columns = SLA_New_Names)

    

    def SLA_df(self):
         self.drop_null_rows()
         self.drop_total_row()
         self.cols_transfomer()


         df = self.df
         return df
        

In [ ]:
class SLA_Cleaner:
    def __init__(self, df):
        self.df = df
         
    def drop_null_rows(self):
        self.df.dropna(how='all', inplace=True)

    def drop_total_row(self):
        if "No." in self.df.columns:
            self.df.drop(self.df[self.df["No."] == "Total"].index, inplace=True)
            
    def pick_column_by_pattern(self, pattern):
        for pat in pattern:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]
        return None
            
    def SLA_col_subsetting(self):
        capacity_col = self.pick_column_by_pattern(['Capacity'])
        comments_col = self.pick_column_by_pattern(['comment'])
        site_desc_col = self.pick_column_by_pattern(['site', 'description'])
        mrc_col = self.pick_column_by_pattern(['month', 'MRC'])
        
        SLA_Cols = {'Link ID': 'Link ID',
                    'SLA Dates': 'SLA Date',
                    'Last Mile': 'Last Mile',            
                    'Capacity': capacity_col,
                    'Sites/Location': site_desc_col,
                    'MRC': mrc_col,
                    'Comments': comments_col
                    }
        self.df = self.df.loc[:, [col for col in SLA_Cols.values()]]

    def cols_transfomer(self):
        self.SLA_col_subsetting()
        self.excise_duty = 0.15
        self.VAT = 0.16
        # Changing the link id to str depending in the current dtype
        if 'Link ID' in self.df.columns:
            if self.df['Link ID'].dtype == 'float':
                self.df['Link ID'] = self.df['Link ID'].astype('int').astype('str')
            elif self.df['Link ID'].dtype == 'int':
                self.df['Link ID'] = self.df['Link ID'].astype('str')
        
        #Date to dtype Date
        if 'SLA Date' in self.df.columns:
            self.df['SLA Date'] = pd.to_datetime(self.df['SLA Date'])
        
        # Capacity
        if self.capacity_picker() in self.df.columns and self.df[self.capacity_picker()].dtype == 'object':
            self.df[self.capacity_picker()] = self.df[self.capacity_picker()].str.split(' ',expand =True)[0].astype('int')
        
        # Calculating the Quarterly Recurring Charges
        if self.MRC_picker() in self.df.columns:
            self.df['QRC'] = np.multiply(np.multiply(self.df[self.MRC_picker()],
                                                (1 + self.excise_duty + self.VAT)),3)
        
        #renaming the Columns
        SLA_New_Names = {
            'Link ID': 'Link_ID',
            'SLA Date': 'SLA_Date',
            'Last Mile': 'Last_Mile',
            'QRC': 'QRC_Incl',
            site_desc_col : 'Location',
            self.capacity_picker(): 'Capacity',
            self.MRC_picker(): 'MRC_Excl',
            self.comments_picker(): 'SLM_Comments'
        }
        self.df = self.df.rename(columns = SLA_New_Names)

    def SLA_df(self):
        self.drop_null_rows()
        self.drop_total_row()
        self.cols_transfomer()

        return self.df


In [77]:
saf_cleaned = SLA_Cleaner(df_saf).SLA_df()
ltk_cleaned = SLA_Cleaner(df_ltk).SLA_df()

In [78]:
saf_cleaned.tail()

,Link_ID,SLA_Date,Last_Mile,Capacity,Location,MRC_Excl,SLM_Comments,QRC_Incl
54,95096538,2023-10-01,MPLS,20,KRA - BUSIA-FIBRE,82200.00,NaN,323046.0000
55,95006961,2023-10-01,MPLS,15,KRA - FIXED 30th Floor 3001,11274.36,NaN,44308.2348
56,95006963,2023-10-01,Internet,15,KRA - FIXED 25th Floor 2575,11274.36,NaN,44308.2348
57,95021939,2023-10-01,Internet,10,KENYA SCHOOL OF REVENUE -FIBER,15300.00,NaN,60129.0000
58,95018756,2023-10-01,MPLS,10,KRA -SIAYA,38000.00,NaN,149340.0000


In [79]:
ltk_cleaned.head()

,Link_ID,SLA_Date,Last_Mile,Capacity,Location,MRC_Excl,SLM_Comments,QRC_Incl
0,1750,2020-10-01,Fibre,90,KRA - TIMES TOWERS TO KRA - KIXP (EADC),270000.0,"16/04/2022: Upgraded 90 Mbps @ KES 649,050 ->...",1061100.000
1,5277,2020-10-01,Fibre,75,KRA CUSTOMS HSE TO KRA TIMES TOWER,179718.0,Not in Invoice/Renamed/Relocated,706291.740
2,5611,2020-10-01,Fibre,8,KRA AIRPORT CONTAINER DEPOT TO CUSTOMS HSE MSA,20642.5,Okay. No change,81125.025
3,20103,2020-10-01,Fibre,9,KRA Wilson Airport Customs to Times Towers - V...,22950.0,Okay. No change,90193.500
4,20954,2020-10-01,Fibre,20,KRA TIMES TOWERS NBO TO EGERTON UNIVERSITY NJO...,50000.0,Okay. No change,196500.000


In [23]:
df_saf.tail()

,No.,KRA SITES,Last Mile,Link ID,Capacity,Monthly Ex,SLA Date,Quarterly Exc,Excise,VAT,Total Quarterly Inc,Comments
54,55,KRA - BUSIA-FIBRE,MPLS,95096538.0,20 Mbps,82200.00,2023-10-01,246600.00,49320.000,47347.20000,343267.20000,NaN
55,56,KRA - FIXED 30th Floor 3001,MPLS,95006961.0,15 Mbps,11274.36,2023-10-01,33823.08,6764.616,6494.03136,47081.72736,NaN
56,57,KRA - FIXED 25th Floor 2575,Internet,95006963.0,15 Mbps,11274.36,2023-10-01,33823.08,6764.616,6494.03136,47081.72736,NaN
57,58,KENYA SCHOOL OF REVENUE -FIBER,Internet,95021939.0,10 Mbps,15300.00,2023-10-01,45900.00,9180.000,8812.80000,63892.80000,NaN
58,59,KRA -SIAYA,MPLS,95018756.0,10 Mbps,38000.00,2023-10-01,114000.00,22800.000,21888.00000,158688.00000,NaN


In [5]:
saf_cleaned.head(2)

,No.,KRA SITES,Last Mile,Link ID,Capacity,Monthly Ex,SLA Date,Quarterly Exc,Excise,VAT,Total Quarterly Inc,Comments
0,1,BUNGOMA MICROWAVE,MPLS,95054463.0,25 Mbps,20352.8,2020-12-16,61058.4,12211.68,11723.2128,84993.2928,Upgrade 5>25 Mbps. Check Upgrade Docs. KES 449...
1,2,BUSIA-WIMAX,MPLS,95096538.0,12 Mbps,70008.3,2020-12-16,210024.9,42004.98,40324.7808,292354.6608,NaN


In [6]:
ltk_cleaned.head(2)

,No.,SLA Service,Link Description,Link ID,Capacity (Mbps),MRC,QRC,Last Mile,SLA Date,Status,SLM Comments
0,1,KRA KIXP,KRA - TIMES TOWERS TO KRA - KIXP (EADC),1750,90,270000.0,810000.0,Fibre,2020-10-01,Active,"16/04/2022: Upgraded 90 Mbps @ KES 649,050 ->..."
1,2,KRA Customs Hse - Times Tower,KRA CUSTOMS HSE TO KRA TIMES TOWER,5277,75,179718.0,539154.0,Fibre,2020-10-01,Active,Not in Invoice/Renamed/Relocated


In [80]:
ltk_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Link_ID       26 non-null     object        
 1   SLA_Date      23 non-null     datetime64[ns]
 2   Last_Mile     26 non-null     object        
 3   Capacity      26 non-null     int64         
 4   Location      24 non-null     object        
 5   MRC_Excl      26 non-null     float64       
 6   SLM_Comments  26 non-null     object        
 7   QRC_Incl      26 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 1.8+ KB


In [22]:
def capacity_picker(df):
    for pat in ['Capacity']:
       matching_cols = [col for col in df.columns if pat.lower() in col.lower()]
       if matching_cols:
           return matching_cols[0]
def comments_picker(df):
    for pat in ['comment']:
        matching_cols = [col for col in df.columns if pat.lower() in col.lower()]
        if matching_cols:
            return matching_cols[0]
def Site_desc_picker(df):
    for pat in ['site', 'description']:
        matching_cols = [col for col in df.columns if pat.lower() in col.lower()]
        if matching_cols:
            return matching_cols[0]
def MRC_picker(df):
    for pat in ['month', 'MRC']:
        matching_cols = [col for col in df.columns if pat.lower() in col.lower()]
        if matching_cols:
            return matching_cols[0]

In [27]:
df= df_saf
SLA_Cols = {'Link ID': 'Link ID',
            'SLA Dates':'SLA Date',
            'Last Mile': 'Last Mile',            
            'Capacity': capacity_picker(df),
            'Sites/Location': Site_desc_picker(df),
            'MRC': MRC_picker(df),
            'Comments':comments_picker(df)}


In [28]:
for i,j in SLA_Cols.items():
    print(f'{i}: {j}')

Link ID: Link ID
SLA Dates: SLA Date
Last Mile: Last Mile
Capacity: Capacity
Sites/Location: KRA SITES
MRC: Monthly Ex
Comments: Comments


In [21]:
comments_picker(df_ltk)


'SLM Comments'

In [13]:
capacity_picker(df_ltk)

'Capacity (Mbps)'

In [37]:
saf_cleaned.head()

,Link ID,SLA Date,Last Mile,Capacity,KRA SITES,Monthly Ex,Comments
0,95054463.0,2020-12-16,MPLS,25 Mbps,BUNGOMA MICROWAVE,20352.8,Upgrade 5>25 Mbps. Check Upgrade Docs. KES 449...
1,95096538.0,2020-12-16,MPLS,12 Mbps,BUSIA-WIMAX,70008.3,NaN
2,95038953.0,2020-12-16,MPLS,80 Mbps,DR BACKHAUL FIBRE,93504.6,NaN
3,95054464.0,2020-12-16,MPLS,60 Mbps,ELDORET KIPTAGICH MPLS FIBER,25439.4,Upgrade 10>60 Mbps. Check Upgrade Docs. KES 69...
4,95054465.0,2020-12-16,MPLS,5 Mbps,ELDORET KPC LOOP MICROWAVE,20357.4,NaN


In [38]:
saf_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Link ID     59 non-null     float64
 1   SLA Date    59 non-null     object 
 2   Last Mile   59 non-null     object 
 3   Capacity    59 non-null     object 
 4   KRA SITES   59 non-null     object 
 5   Monthly Ex  59 non-null     float64
 6   Comments    21 non-null     object 
dtypes: float64(2), object(5)
memory usage: 3.4+ KB


In [60]:
import numpy as np
import pandas as pd
def cols_transfomer(df):
    excise_duty = 0.15
    VAT = 0.16
    # Changing the link id to str depending in the current dtype
    if 'Link ID' in df.columns:
        if df['Link ID'].dtype == 'float':
            df['Link ID'] = df['Link ID'].astype('int').astype('str')
        elif df['Link ID'].dtype == 'int':
            df['Link ID'] = df['Link ID'].astype('str')
    
    #Date to dtype Date
    if 'SLA Date' in df.columns:
        df['SLA Date'] = pd.to_datetime(df['SLA Date'])
    
    # Capacity
    if capacity_picker(df) in df.columns and df[capacity_picker(df)].dtype == 'object':
        df[capacity_picker(df)] = df[capacity_picker(df)].str.split(' ',expand =True)[0].astype('int')
    
    # Calculating the Quarterly Recurring Charges
    if MRC_picker(df) in df.columns:
        df['QRC'] = np.multiply(np.multiply(df[MRC_picker(df)],
                                            (1 + excise_duty + VAT)),3)
        
    #renaming the Columns
    SLA_New_Names = {
        'Link ID': 'Link_ID',
        'SLA Date': 'SLA_Date',
        'Last Mile': 'Last_Mile',
        'QRC': 'QRC_Incl',
        Site_desc_picker(df): 'Location',
        capacity_picker(df): 'Capacity',
        MRC_picker(df): 'MRC_Excl',
        comments_picker(df): 'SLM_Comments'
    }
    df = df.rename(columns = SLA_New_Names)

    return df


In [61]:
df = ltk_cleaned
saf_clean = cols_transfomer(df)

In [62]:
saf_clean.head()

,Link_ID,SLA_Date,Last_Mile,Capacity,Location,MRC_Excl,SLM_Comments,QRC_Incl
0,1750,2020-10-01,Fibre,90,KRA - TIMES TOWERS TO KRA - KIXP (EADC),270000.0,"16/04/2022: Upgraded 90 Mbps @ KES 649,050 ->...",1061100.000
1,5277,2020-10-01,Fibre,75,KRA CUSTOMS HSE TO KRA TIMES TOWER,179718.0,Not in Invoice/Renamed/Relocated,706291.740
2,5611,2020-10-01,Fibre,8,KRA AIRPORT CONTAINER DEPOT TO CUSTOMS HSE MSA,20642.5,Okay. No change,81125.025
3,20103,2020-10-01,Fibre,9,KRA Wilson Airport Customs to Times Towers - V...,22950.0,Okay. No change,90193.500
4,20954,2020-10-01,Fibre,20,KRA TIMES TOWERS NBO TO EGERTON UNIVERSITY NJO...,50000.0,Okay. No change,196500.000


In [63]:
saf_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Link_ID       26 non-null     object        
 1   SLA_Date      23 non-null     datetime64[ns]
 2   Last_Mile     26 non-null     object        
 3   Capacity      26 non-null     int64         
 4   Location      24 non-null     object        
 5   MRC_Excl      26 non-null     float64       
 6   SLM_Comments  26 non-null     object        
 7   QRC_Incl      26 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 1.8+ KB


## Data Cleaner Class 

In [85]:
import pandas as pd
import numpy as np

class DataCleaner:
    def __init__(self, df):
        self.df = df

    def drop_null_rows(self):
        """Drop rows with all null values."""
        self.df.dropna(how='all', inplace=True)

    def drop_total_row(self):
        """Drop rows where 'No.' column contains 'Total'."""
        if "No." in self.df.columns:
            self.df.drop(self.df[self.df["No."] == "Total"].index, inplace=True)

    def pick_invoice_column_name(self):
        """Pick the column name containing invoice numbers."""
        for substring in ['invoice no', 'ref']:
            matching_columns = [col for col in self.df.columns if substring.lower() in col.lower()]
            if matching_columns:
                return matching_columns[0]

    def QRC_Finder(self):
        """Find the column with Quarterly Recurring Charges."""
        max_value = None
        max_col = None
        for substring in ['QRC', 'Total', 'Amount']:
            fields = [col for col in self.df.columns if substring.lower() in col.lower()]
            for field in fields:
                if field in self.df.columns and not self.df[field].isnull().all():
                    first_row_value = self.df.at[1, field]
                    if max_value is None or first_row_value > max_value:
                        max_value = first_row_value
                        max_col = field
        return max_col

    def clean_invoice_data(self):
        """Clean invoice data."""
        self.drop_null_rows()
        self.drop_total_row()
        self.rename_invoice_columns()
        self.adjust_link_id_dtype()

    def rename_invoice_columns(self):
        """Rename invoice columns."""
        columns_wanted = {
            "Date": "Invoice Date",
            "ID": "Link ID",
            "Period": "Invoice Period",
            "Description": "Invoice Description",
            "Reference/No": self.pick_invoice_column_name(),
            "QRC": self.QRC_Finder()
        }
        self.df = self.df.loc[:, [value for value in columns_wanted.values()]]

        new_invoice_col_names = {
            "Invoice Date": "Invoice_Data",
            "Link ID": "Link_ID",
            "Invoice Period": "Invoice_Period",
            "Invoice Description": "Invoice_Description",
            self.pick_invoice_column_name(): "Invoice_Reference",
            self.QRC_Finder(): "Total_QRC"
        }
        self.df = self.df.rename(columns=new_invoice_col_names)

    def adjust_link_id_dtype(self):
        """Adjust 'Link ID' column data type."""
        if 'Link ID' in self.df.columns:
            if self.df['Link ID'].dtype == 'float':
                self.df['Link ID'] = self.df['Link ID'].astype('int').astype('str')
            elif self.df['Link ID'].dtype == 'int':
                self.df['Link ID'] = self.df['Link ID'].astype('str')

    def pick_sla_column_name(self):
        """Pick the column name for SLA data."""
        for pat in ['Capacity']:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]
    def capacity_picker(self):
        for pat in ['Capacity']:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]

    def comments_picker(self):
        """Pick the column containing comments."""
        for pat in ['comment']:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]

    def Site_desc_picker(self):
        """Pick the column containing site descriptions."""
        for pat in ['site', 'description']:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]

    def MRC_picker(self):
        """Pick the column containing MRC (Monthly Recurring Charges)."""
        for pat in ['month', 'MRC']:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]

    def SLA_col_subsetting(self):
        """Subset SLA columns."""
        self.capacity_picker()
        self.comments_picker()
        self.Site_desc_picker()
        self.MRC_picker()

        SLA_Cols = {
            'Link ID': 'Link ID',
            'SLA Dates': 'SLA Date',
            'Last Mile': 'Last Mile',
            'Capacity': self.capacity_picker(),
            'Sites/Location': self.Site_desc_picker(),
            'MRC': self.MRC_picker(),
            'Comments': self.comments_picker()
        }
        self.df = self.df.loc[:, [col for col in SLA_Cols.values()]]

    def transform_sla_data(self):
        """Transform SLA data."""
        self.SLA_col_subsetting()
        self.excise_duty = 0.15
        self.VAT = 0.16

        # Adjust 'Link ID' data type
        if 'Link ID' in self.df.columns:
            if self.df['Link ID'].dtype == 'float':
                self.df['Link ID'] = self.df['Link ID'].astype('int').astype('str')
            elif self.df['Link ID'].dtype == 'int':
                self.df['Link ID'] = self.df['Link ID'].astype('str')

        # Convert 'SLA Date' to datetime
        if 'SLA Date' in self.df.columns:
            self.df['SLA Date'] = pd.to_datetime(self.df['SLA Date'])

        # Extract numeric values from 'Capacity' column
        if self.capacity_picker() in self.df.columns and self.df[self.capacity_picker()].dtype == 'object':
            self.df[self.capacity_picker()] = self.df[self.capacity_picker()].str.split(' ', expand=True)[0].astype('int')

        # Calculate Quarterly Recurring Charges (QRC)
        if self.MRC_picker() in self.df.columns:
            self.df['QRC'] = np.multiply(np.multiply(self.df[self.MRC_picker()], (1 + self.excise_duty + self.VAT)), 3)

        # Rename columns
        SLA_New_Names = {
            'Link ID': 'Link_ID',
            'SLA Date': 'SLA_Date',
            'Last Mile': 'Last_Mile',
            'QRC': 'QRC_Incl',
            self.Site_desc_picker(): 'Location',
            self.capacity_picker(): 'Capacity',
            self.MRC_picker(): 'MRC_Excl',
            self.comments_picker(): 'SLM_Comments'
        }
        self.df = self.df.rename(columns=SLA_New_Names)

    def clean_sla_data(self):
        """Clean SLA data."""
        self.drop_null_rows()
        self.drop_total_row()
        self.transform_sla_data()

    def invoice_df(self):
        """Return cleaned invoice data."""
        self.clean_invoice_data()
        return self.df

    def sla_df(self):
        """Return cleaned SLA data."""
        self.clean_sla_data()
        return self.df


In [91]:
saf_cleaned = DataCleaner(df_saf).sla_df()
saf_cleaned2 = DataCleaner(df_saf).invoice_df()


KeyError: "['Invoice Date', 'Invoice Period', 'Invoice Description', None] not in index"

In [88]:
saf_cleaned.head()

,Link_ID,SLA_Date,Last_Mile,Capacity,Location,MRC_Excl,SLM_Comments,QRC_Incl
0,95054463,2020-12-16,MPLS,25,BUNGOMA MICROWAVE,20352.8,Upgrade 5>25 Mbps. Check Upgrade Docs. KES 449...,79986.504
1,95096538,2020-12-16,MPLS,12,BUSIA-WIMAX,70008.3,NaN,275132.619
2,95038953,2020-12-16,MPLS,80,DR BACKHAUL FIBRE,93504.6,NaN,367473.078
3,95054464,2020-12-16,MPLS,60,ELDORET KIPTAGICH MPLS FIBER,25439.4,Upgrade 10>60 Mbps. Check Upgrade Docs. KES 69...,99976.842
4,95054465,2020-12-16,MPLS,5,ELDORET KPC LOOP MICROWAVE,20357.4,NaN,80004.582


In [90]:
saf_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Link_ID       59 non-null     object        
 1   SLA_Date      59 non-null     datetime64[ns]
 2   Last_Mile     59 non-null     object        
 3   Capacity      59 non-null     int32         
 4   Location      59 non-null     object        
 5   MRC_Excl      59 non-null     float64       
 6   SLM_Comments  21 non-null     object        
 7   QRC_Incl      59 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int32(1), object(4)
memory usage: 3.6+ KB


In [92]:
import pandas as pd
import numpy as np

class InvoiceCleaner:
    def __init__(self, df):
        self.df = df

    def drop_null_rows(self):
        self.df.dropna(how='all', inplace=True)

    def drop_total_row(self):
        if "No." in self.df.columns:
            self.df.drop(self.df[self.df["No."] == "Total"].index, inplace=True)

    def pick_invoice_column_name(self):
        for substring in ['invoice no', 'ref']:
            matching_columns = [col for col in self.df.columns if substring.lower() in col.lower()]
            if matching_columns:
                return matching_columns[0]

    def QRC_Finder(self):
        max_value = None
        max_col = None
        for substring in ['QRC', 'Total', 'Amount']:
            fields = [col for col in self.df.columns if substring.lower() in col.lower()]
            for field in fields:
                if field in self.df.columns and not self.df[field].isnull().all():
                    first_row_value = self.df.at[1, field]
                    if max_value is None or first_row_value > max_value:
                        max_value = first_row_value
                        max_col = field
        return max_col

    def clean_data(self):
        self.drop_null_rows()
        self.drop_total_row()

        columns_wanted = {
            "Date": "Invoice Date",
            "ID": "Link ID",
            "Period": "Invoice Period",
            "Description": "Invoice Description",
            "Reference/No": self.pick_invoice_column_name(),
            "QRC": self.QRC_Finder()
        }

        self.df = self.df.loc[:, [value for value in columns_wanted.values()]]

        new_invoice_col_names = {
            "Invoice Date": "Invoice_Data",
            "Link ID": "Link_ID",
            "Invoice Period": "Invoice_Period",
            "Invoice Description": "Invoice_Description",
            self.pick_invoice_column_name(): "Invoice_Reference",
            self.QRC_Finder(): "Total_QRC"
        }
        self.df = self.df.rename(columns=new_invoice_col_names)

        # Adjust 'Link ID' based on its current type
        if 'Link_ID' in self.df.columns:
            if self.df['Link_ID'].dtype == 'float':
                self.df['Link_ID'] = self.df['Link_ID'].astype('int').astype('str')
            elif self.df['Link_ID'].dtype == 'int':
                self.df['Link_ID'] = self.df['Link_ID'].astype('str')

        # Convert 'Invoice Date' to datetime
        if 'Invoice_Data' in self.df.columns:
            self.df['Invoice_Data'] = pd.to_datetime(self.df['Invoice_Data'])

        return self.df


class SLACleaner:
    def __init__(self, df):
        self.df = df

    def drop_null_rows(self):
        self.df.dropna(how='all', inplace=True)

    def drop_total_row(self):
        if "No." in self.df.columns:
            self.df.drop(self.df[self.df["No."] == "Total"].index, inplace=True)

    def capacity_picker(self):
        for pat in ['Capacity']:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]

    def comments_picker(self):
        for pat in ['comment']:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]

    def site_desc_picker(self):
        for pat in ['site', 'description']:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]

    def MRC_picker(self):
        for pat in ['month', 'MRC']:
            matching_cols = [col for col in self.df.columns if pat.lower() in col.lower()]
            if matching_cols:
                return matching_cols[0]

    def clean_data(self):
        self.drop_null_rows()
        self.drop_total_row()

        SLA_Cols = {
            'Link ID': 'Link ID',
            'SLA Dates': 'SLA Date',
            'Last Mile': 'Last Mile',
            'Capacity': self.capacity_picker(),
            'Sites/Location': self.site_desc_picker(),
            'MRC': self.MRC_picker(),
            'Comments': self.comments_picker()
        }

        self.df = self.df.loc[:, [col for col in SLA_Cols.values()]]

        # Changing the link id to str depending on the current dtype
        if 'Link ID' in self.df.columns:
            if self.df['Link ID'].dtype == 'float':
                self.df['Link ID'] = self.df['Link ID'].astype('int').astype('str')
            elif self.df['Link ID'].dtype == 'int':
                self.df['Link ID'] = self.df['Link ID'].astype('str')

        # Convert 'SLA Date' to datetime
        if 'SLA Date' in self.df.columns:
            self.df['SLA Date'] = pd.to_datetime(self.df['SLA Date'])

        # Capacity
        if self.capacity_picker() in self.df.columns and self.df[self.capacity_picker()].dtype == 'object':
            self.df[self.capacity_picker()] = self.df[self.capacity_picker()].str.split(' ', expand=True)[0].astype(
                'int')

        # Calculating the Quarterly Recurring Charges
        if self.MRC_picker() in self.df.columns:
            self.excise_duty = 0.15
            self.VAT = 0.16
            self.df['QRC'] = np.multiply(np.multiply(self.df[self.MRC_picker()], (1 + self.excise_duty + self.VAT)), 3)

        # Renaming the Columns
        SLA_New_Names = {
            'Link ID': 'Link_ID',
            'SLA Date': 'SLA_Date',
            'Last Mile': 'Last_Mile',
            'QRC': 'QRC_Incl',
            self.site_desc_picker(): 'Location',
            self.capacity_picker(): 'Capacity',
            self.MRC_picker(): 'MRC_Excl',
            self.comments_picker(): 'SLM_Comments'
        }
        self.df = self.df.rename(columns=SLA_New_Names)

        return self.df


In [97]:
saf_clean = SLACleaner(df_saf).clean_data()
saf_clean2 = InvoiceCleaner(df_saf).clean_data()

KeyError: "['Invoice Date', 'Invoice Period', 'Invoice Description', None] not in index"

In [95]:
saf_clean.head()

,Link_ID,SLA_Date,Last_Mile,Capacity,Location,MRC_Excl,SLM_Comments,QRC_Incl
0,95054463,2020-12-16,MPLS,25,BUNGOMA MICROWAVE,20352.8,Upgrade 5>25 Mbps. Check Upgrade Docs. KES 449...,79986.504
1,95096538,2020-12-16,MPLS,12,BUSIA-WIMAX,70008.3,NaN,275132.619
2,95038953,2020-12-16,MPLS,80,DR BACKHAUL FIBRE,93504.6,NaN,367473.078
3,95054464,2020-12-16,MPLS,60,ELDORET KIPTAGICH MPLS FIBER,25439.4,Upgrade 10>60 Mbps. Check Upgrade Docs. KES 69...,99976.842
4,95054465,2020-12-16,MPLS,5,ELDORET KPC LOOP MICROWAVE,20357.4,NaN,80004.582


In [96]:
saf_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Link_ID       59 non-null     object        
 1   SLA_Date      59 non-null     datetime64[ns]
 2   Last_Mile     59 non-null     object        
 3   Capacity      59 non-null     int32         
 4   Location      59 non-null     object        
 5   MRC_Excl      59 non-null     float64       
 6   SLM_Comments  21 non-null     object        
 7   QRC_Incl      59 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int32(1), object(4)
memory usage: 3.6+ KB


In [2]:
import pandas as pd
ltk_sla_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\LTK_SLA.csv')
saf_sla_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\SAFARICOM_SLA.csv')

ltk_invoice_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\LTK_Invoice.csv')
saf_invoice_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\SAFARICOM_Invoice.csv')

In [4]:
# Trying to append the data
df = pd.concat([ltk_invoice_df,saf_invoice_df])

In [5]:
df.head()

,Invoice_Data,Link_ID,Invoice_Period,Invoice_Description,Invoice_Reference,Total_QRC,Service_Provider
0,2023-10-13,26046,01-Oct-2023 to 31-Dec-2023,26046 National MPLS KRA MBITA TO VRF (INS-57),CRN-15473,-55986.24,LTK
1,2023-10-06,C-00159-0087,01-Oct-2023 to 31-Dec-2023,C-00159-0087 EPL Connection charge 80MBPS->SAM...,116426,417600.00,LTK
2,2023-10-06,1750,01-Oct-2023 to 31-Dec-2023,01750 National MPLS KRA - TIMES TOWERS TO VRF,116426,939600.00,LTK
3,2023-10-06,5611,01-Oct-2023 to 31-Dec-2023,05611 National MPLS Msa link vlan 460 customs,116426,71835.90,LTK
4,2023-10-06,20103,01-Oct-2023 to 31-Dec-2023,20103 Leased_Circuits_Naticnal KRA WILSON AIRP...,116426,79866.00,LTK


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84 entries, 0 to 57
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Invoice_Data         84 non-null     object 
 1   Link_ID              84 non-null     object 
 2   Invoice_Period       84 non-null     object 
 3   Invoice_Description  84 non-null     object 
 4   Invoice_Reference    84 non-null     object 
 5   Total_QRC            84 non-null     float64
 6   Service_Provider     84 non-null     object 
dtypes: float64(1), object(6)
memory usage: 5.2+ KB


In [1]:
import pandas as pd
import numpy as np
str1 = 'Jackrony Karani '
str2 = str1

In [3]:
str1.strip().lower()

'jackrony karani'

In [1]:
!pip install mysql-connector-python


  Using cached mysql_connector_python-8.3.0-cp310-cp310-win_amd64.whl.metadata (2.0 kB)
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB 145.2 kB/s eta 0:01:46
   ---------------------------------------- 0.0/15.4 MB 145.2 kB/s eta 0:01:46
   ---------------------------------------- 0.1/15.4 MB 233.8 kB/s eta 0:01:06
   ---------------------------------------- 0.1/15.4 MB 399.4 kB/s eta 0:00:39
    --------------------------------------- 0.2/15.4 MB 719.7 kB/s eta 0:00:22
    --------------------------------------- 0.3/15.4 MB 873.8 kB/s eta 0:00:18
   - -------------------------------------- 0.5/15.4 MB 1.1 MB/s eta 0:00:15
   -- ------------------------------------- 0.8/15.4 MB 1.7 MB/s eta 0:00:09
   --- ------------------------------------ 1.2/15.4 MB 2.2 MB/s eta 0:00:

In [1]:
import pandas as pd
ltk_sla_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\LTK_SLA.csv')
saf_sla_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\SAFARICOM_SLA.csv')
combined_sla = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\SLA_Combined.csv')

ltk_invoice_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\LTK_Invoice.csv')
saf_invoice_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\SAFARICOM_Invoice.csv')
combined_Invoice = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\Invoice_Combined.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\SLA_ETL\\Processsed_Data\\SAFARICOM_SLA.csv'

In [ ]:
# Filter rows where Link_ID count is greater than 2
duplicate_link_ids = combined_Invoice['Link_ID'].value_counts()[lambda x: x > 1].index

duplicate_link_ids

In [ ]:
def duplicate_handler(self):
    links_with_dupl = self.df['Link_ID'].value_counts()[lambda x: x > 1].index
    for id in links_with_dupl:
        self.df.drop(self.df[(self.df['Link_ID'] == id) & (self.df['Total_QRC'] == self.df.groupby(by = LID)['Total_QRC'].mean ],axis = 0)

In [ ]:
def invoice_duplicate_handler(self):
    # Identify duplicate Link_IDs
    duplicate_link_ids = self.df[self.df.duplicated(subset='Link_ID', keep=False)]['Link_ID'].unique()
    
    # Iterate over each duplicate Link ID
    for link_id in duplicate_link_ids:
        # Filter the dataframe for all occurrences of the current Link ID
        duplicates = self.df[self.df['Link_ID'] == link_id]
        
        # Find the index of the row with the highest Total_QRC
        max_qrc_index = duplicates['Total_QRC'].idxmax()
        
        # Drop all other duplicates except for the one with the highest Total_QRC
        self.df = self.df.drop(duplicates.index.difference([max_qrc_index]))


In [ ]:
def invocie_duplicate_handler(df):
    # Identify duplicate Link_IDs
    duplicate_link_ids = df[df.duplicated(subset='Link_ID', keep=False)]['Link_ID'].unique()
    
    # Iterate over each duplicate Link ID
    for link_id in duplicate_link_ids:
        # Filter the dataframe for all occurrences of the current Link ID
        duplicates = df[df['Link_ID'] == link_id]
        
        # Find the index of the row with the highest Total_QRC
        max_qrc_index = duplicates['Total_QRC'].idxmax()
        
        # Drop all other duplicates except for the one with the highest Total_QRC
        df = df.drop(duplicates.index.difference([max_qrc_index]))

        return df

### SLA Duplicates Hundler

duplicates_IDs = df['Link_ID'].value_counts[lambda x: x >1].index
df[df['Link_ID'].isin(duplicates_IDs)]

In [2]:
def sla_duplicates_handler(self):
    #Duplicate Link IDS
    duplicate_link_ids = self.df['Link_ID'].value_counts()[lambda x:x>1].index

    for id in duplicate_link_ids:
        #Subsetting the duplicate Data set
        duplicates_df = self.df[self.df['Link_ID'] == id]

        #Finding Index of row with theh highest capacity..We want to maintain the upgraded Link
        max_cap_index = duplicates_df['Capacity(Mbps)'].idxmax()

        #Droppping all otherduplicates exce[t the one with upgraded capacity
        self.df = self.df.drop(duplicates_df.index.difference([max_cap_index]))

    return self.df

In [7]:
def sla_duplicates_handler(df):
    #Duplicate Link IDS
    duplicate_link_ids = df['Link_ID'].value_counts()[lambda x:x>1].index

    for id in duplicate_link_ids:
        #Subsetting the duplicate Data set
        duplicates_df = df[df['Link_ID']== id]

        #Finding Index of row with theh highest capacity..We want to maintain the upgraded Link
        max_cap_index = duplicates_df['Capacity(Mbps)'].idxmax()

        #Droppping all otherduplicates exce[t the one with upgraded capacity
        df = df.drop(duplicates_df.index.difference([max_cap_index]))

    return df

In [8]:
sla_dup_free = sla_duplicates_handler(combined_sla)

In [ ]:
sla_dup_free

In [3]:
import pymysql

In [5]:
import logging
import ujson

import apache_beam as beam
from apache_beam.runners import PipelineState
from apache_beam.io.gcp.bigquery import BigQueryDisposition
from apache_beam.io.gcp.internal.clients.bigquery import TableFieldSchema

from pipe_tools.generator import MessageGenerator
from pipe_tools.generator import GenerateMessages
from pipe_tools.io import WriteToBigQueryDatePartitioned
from pipe_tools.coders import JSONDictCoder

from pipe_template.options import TemplateOptions
from pipe_template.transform import AddField


def run(options):

    template_options = options.view_as(TemplateOptions)

    source = GenerateMessages(generator=MessageGenerator())
    sink = beam.io.WriteToText(file_path_prefix=template_options.dest, coder=JSONDictCoder())

    pipeline = beam.Pipeline(options=options)
    (
        pipeline
        | "generate" >> source
        | "tag" >> beam.ParDo(AddField(field=template_options.tag_field,
                                       value=template_options.tag_value))
        | "write" >> sink
    )

    result = pipeline.run()
    success_states = set([PipelineState.DONE])

    if template_options.wait:
        result.wait_until_finish()
    else:
        success_states.add(PipelineState.RUNNING)
        success_states.add(PipelineState.UNKNOWN)

    logging.info('returning with result.state=%s' % result.state)
    return 0 if result.state in success_states else 1

ImportError: cannot import name 'TableFieldSchema' from 'apache_beam.io.gcp.internal.clients.bigquery' (c:\Users\USER\anaconda3\envs\Python_Dev\lib\site-packages\apache_beam\io\gcp\internal\clients\bigquery\__init__.py)

In [3]:
#%pip install pep517
%pip install dotenv
#import pep517

  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [76 lines of output]
      c:\Users\USER\anaconda3\envs\Python_Dev\lib\site-packages\setuptools\__init__.py:80: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
        error: subprocess-exited-with-error
      
        Ã— python setup.py egg_info did not run successfully.
        â”‚ exit code: 1
        â•°â”€> [16 lines of output]
            Traceback (most recent call last):
              File "<string>", line 2, in <modu

In [1]:
import dotenv

ModuleNotFoundError: No module named 'dotenv'

In [4]:
!python -m pip install --upgrade pip setuptools


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 326.8 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.1 MB 585.1 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.1 MB 876.1 kB/s eta 0:00:03
   ----- ---------------------------------- 0.3/2.1 MB 1.1 MB/s eta 0:00:02
   ------ --------------------------------- 0.4/2.1 MB 1.2 MB/s eta 0:00:02
   ---------- ----------------------------- 0.6/2.1 MB 1.5 MB/s eta 0:00:02
   -------------- ------------------------- 0.7/2.1 MB 2.0 MB/s eta 0:00:01
   -------------- ------------------------- 0.7/2.1 MB 2.0 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 3.7 MB/s eta 0:00:00
   ------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
#Installing the dotenv using pep517
%pip install --use-pep517 dotenv

  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [35 lines of output]
        Using cached distribute-0.7.3.zip (145 kB)
        Preparing metadata (setup.py): started
        Preparing metadata (setup.py): finished with status 'error'
        error: subprocess-exited-with-error
      
        Ã— python setup.py egg_info did not run successfully.
        â”‚ exit code: 1
        â•°â”€> [16 lines of output]
            Traceback (most recent call last):
              File "<string>", line 2, in <module>
              File "<pip-setuptools-caller>", line 14, in <module>
              File "C:\Users\USER\AppData\Local\Temp\pip-install-c43zdcgl\distribute_749795dbe8ce4461893425c91eb3f39e\setuptools\__init__.py", line 2, in <module>
                from setuptools.extension import Extension, Library
              File "C:\Users\USER\AppData\Local\Temp\pip-install-c43zdcgl\distribute_749795dbe8ce4461

In [2]:
%pip cache purge
%pip install dotenv


Files removed: 397
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [76 lines of output]
      c:\Users\USER\anaconda3\envs\Python_Dev\lib\site-packages\setuptools\__init__.py:81: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
        error: subprocess-exited-with-error
      
        Ã— python setup.py egg_info did not run successfully.
        â”‚ exit code: 1
        â•°â”€> [16 lines of output]
            Traceback (most recent call last):
              File "<string>", line 2, in <modu

In [1]:
import dotenv

In [5]:
import os
from dotenv import load_dotenv


load_dotenv("SLA DP envionment file.env")

ed = os.getenv('excise_duty')

print(f'Raw Data Folder Path: {ed}')
type(ed)

Raw Data Folder Path: 0.15


str

In [7]:
float(ed)

0.15

#### Unique ID

In [5]:
def sp_modifier(sp):
    sp_new_names = {
        'SAFARICOM':'Saf',
        'LTK': 'LTK'
    }
    return sp_new_names[sp]
workbook_name = "SAFARICOM INVOICE"
w_b = sp_modifier(workbook_name.split(" ")[0])

In [6]:
w_b

'Saf'

In [2]:
import pandas as pd
ltk_sla_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\LTK_SLA.csv')
saf_sla_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\Saf_SLA.csv')
combined_sla = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\SLA_Combined.csv')

ltk_invoice_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\LTK_Invoice.csv')
saf_invoice_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\Saf_Invoice.csv')
combined_Invoice = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\Invoice_Combined.csv')

In [8]:
combined_sla['QRC_Incl'].sum()
combined_Invoice['Total_QRC'].sum()

18518106.1816692

In [36]:
pd.to_datetime(saf_sla_df['SLA_Date']).unique()
#saf_sla_df['SLA_Date'].dt.year

def assign_rank(df,sp):
    #Extracting unique values from the date col and sort them
    unique_dates = sorted(df['SLA_Date'].dt.strftime('%Y-%m-%d').unique())

    #creating a mapping of date to rank
    date_to_rank = {date: f'{i:02d}' for i,date in enumerate(unique_dates)}

    #add a new col called rank based on the mapping
    df['rank'] =  df['SLA_Date'].dt.strftime('%Y-%m-%d').map(date_to_rank)



    #Adding the SLA ID Col
    df['SLA_ID'] = df.apply(lambda row: f"{row['SLA_Date'].year}-{row['SLA_Date'].month}-{sp}{row['rank']}", axis=1)
    #df['SLA_ID'] = df.apply(lambda row: f"{row['SLA_Date'].split('-', expand =True)[1]}-{row['SLA_Date'].split('-', expand =True)[1]}-{sp}-{row['rank']}", axis=1)


    return df

In [41]:
saf_sla_df['SLA_Date'] = pd.to_datetime( saf_sla_df['SLA_Date'])
df1 = assign_rank(saf_sla_df,'Saf')
df1['UI'] = df1['SLA_ID'] + "_" + df1['Link_ID'].astype(str)
df1.head()

,Link_ID,SLA_Date,Last_Mile,Capacity_in_Mbps,Location,MRC_Excl,SLM_Comments,QRC_Incl,SLA_ID,Service_Provider,rank,UI
0,95054463,2020-12-16,MPLS,25,BUNGOMA MICROWAVE,20352.80,Upgrade 5>25 Mbps. Check Upgrade Docs. KES 449...,70827.74,2020-12-Saf00,SAFARICOM,00,2020-12-Saf00_95054463
1,95038953,2020-12-16,MPLS,80,DR BACKHAUL FIBRE,93504.60,NaN,325396.01,2020-12-Saf00,SAFARICOM,00,2020-12-Saf00_95038953
2,95054464,2020-12-16,MPLS,60,ELDORET KIPTAGICH MPLS FIBER,25439.40,Upgrade 10>60 Mbps. Check Upgrade Docs. KES 69...,88529.11,2020-12-Saf00,SAFARICOM,00,2020-12-Saf00_95054464
3,95054465,2020-12-16,MPLS,5,ELDORET KPC LOOP MICROWAVE,20357.40,NaN,70843.75,2020-12-Saf00,SAFARICOM,00,2020-12-Saf00_95054465
4,95034216,2020-12-16,Internet,180,HQ-INTERNET,222299.85,NaN,889644.00,2020-12-Saf00,SAFARICOM,00,2020-12-Saf00_95034216


##### Modifying the Invoice to include the SLA Date 
The SLA Date will be used to create SLA_IDS which will help create unique identifier 


In [2]:
import glob
import pandas as pd 
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv("SLA DP envionment file.env")
import xlsxwriter

In [3]:
folder_path = os.getenv('pre_processed_data')

In [4]:
excel_files = glob.glob(f"{folder_path}/*.xlsx")

In [6]:
for file in excel_files:
    xls = pd.ExcelFile(file)
    workbook_name = os.path.splitext(os.path.basename(file))[0]

    for sheet in xls.sheet_names:
        if "sla" in sheet.lower():
            df_sla = pd.read_excel(file, sheet_name= sheet)
        elif "invoice" in sheet.lower():
            df_invoice = pd.read_excel(file, sheet_name=sheet)
    df_invoice= pd.merge(df_invoice, df_sla[['Link ID', 'SLA Date']], on='Link ID', how = 'left')

    # Merging the sheets back into a workbook
    #Creating a pandas excel writer using XlsxWriter as the engine 
    with pd.ExcelWriter(file, engine= 'xlsxwriter', mode='w') as writer:
        df_sla.to_excel(writer, sheet_name = 'SLA', index = False)
        df_invoice.to_excel(writer, sheet_name = 'invoice', index = False)

In [9]:
saf_sla_df.head()
saf_invoice_df.head()
saf_invoice_df['SLA_Date'] = 

,Invoice_Date,Link_ID,Invoice_Period,Invoice_Description,Invoice_Reference,Total_QRC,Unique_Link_ID,Service_Provider
0,2023-10-01,95001135,01-Oct-2023 to 31-Dec-2023,CUSTOMS OFFICE MALINDI-WIMAX,B1-10096515502,240145.6128,B1-10096515502_95001135,Saf
1,2023-10-01,95001574,01-Oct-2023 to 31-Dec-2023,KRA LOITOKTOK BORDER WIMAX,B1-10096515502,240145.6128,B1-10096515502_95001574,Saf
2,2023-10-01,95001628,01-Oct-2023 to 31-Dec-2023,KRA KRATI MOMBASA WIMAX,B1-10096515502,240145.6128,B1-10096515502_95001628,Saf
3,2023-10-01,95004139,01-Oct-2023 to 31-Dec-2023,KRA - MANDERA,B1-10096515502,180078.7944,B1-10096515502_95004139,Saf
4,2023-10-01,95004140,01-Oct-2023 to 31-Dec-2023,KRA - Moyale,B1-10096515502,240145.6128,B1-10096515502_95004140,Saf


In [12]:
saf_invoice_df = pd.merge(saf_invoice_df, saf_sla_df[['Link_ID', 'SLA_Date']], on = 'Link_ID', how='left')

In [13]:
saf_invoice_df.head()

,Invoice_Date,Link_ID,Invoice_Period,Invoice_Description,Invoice_Reference,Total_QRC,Unique_Link_ID,Service_Provider,SLA_Date
0,2023-10-01,95001135,01-Oct-2023 to 31-Dec-2023,CUSTOMS OFFICE MALINDI-WIMAX,B1-10096515502,240145.6128,B1-10096515502_95001135,Saf,2020-12-16
1,2023-10-01,95001574,01-Oct-2023 to 31-Dec-2023,KRA LOITOKTOK BORDER WIMAX,B1-10096515502,240145.6128,B1-10096515502_95001574,Saf,2020-12-16
2,2023-10-01,95001628,01-Oct-2023 to 31-Dec-2023,KRA KRATI MOMBASA WIMAX,B1-10096515502,240145.6128,B1-10096515502_95001628,Saf,2020-12-16
3,2023-10-01,95004139,01-Oct-2023 to 31-Dec-2023,KRA - MANDERA,B1-10096515502,180078.7944,B1-10096515502_95004139,Saf,2020-12-16
4,2023-10-01,95004140,01-Oct-2023 to 31-Dec-2023,KRA - Moyale,B1-10096515502,240145.6128,B1-10096515502_95004140,Saf,2020-12-16


In [9]:
glob.glob('data\*.xlsx')

['data\\LTK SLA & Invoice.xlsx', 'data\\SAFARICOM SLA & Invoice.xlsx']

In [5]:
import glob
import dotenv
import os
dotenv.load_dotenv("SLA DP envionment file.env")
folder_path = os.getenv('raw_data')
excel_files = glob.glob(f"{folder_path}/*.xlsx")

In [6]:
excel_files

['C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\SLA_ETL\\Data\\LTK SLA & Invoice.xlsx',
 'C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\SLA_ETL\\Data\\SAFARICOM SLA & Invoice.xlsx']

In [14]:
for file in excel_files:
    file_path = os.path.splitext(file)[0]
    new_file_path = f'{file_path}.csv'
    print(new_file_path)

C:\Users\USER\OneDrive\KRA WORK\2024 WS\SLA EDA\SLA_ETL\Data\LTK SLA & Invoice.csv
C:\Users\USER\OneDrive\KRA WORK\2024 WS\SLA EDA\SLA_ETL\Data\SAFARICOM SLA & Invoice.csv


In [44]:
def sp_modifier(sp):
    sp_new_names = {
        'SAFARICOM':'Saf',
        'LTK': 'LTK'
    }
    if sp in [key for key, _ in sp_new_names.items()]:
        return sp_new_names[sp]
    else:
        return sp

sp_modifier('ltk')

'ltk'

In [21]:
import numpy as np

In [33]:
sp_new_names = {
    'SAFARICOM':'Saf',
    'LTK': 'LTK'
}

for key, _ in sp_new_names.items():
    print (key.lower())

    

safaricom
ltk


In [37]:
sp = 'ltk'
if sp.lower() in [key.lower() for key,_ in sp_new_names.items()]:
    print('Yes')

Yes


In [48]:
import pandas as pd
 ltk_invoice_df['SLA_Date'].dt.year

In [52]:
pd.to_datetime(ltk_invoice_df['SLA_Date']).dt.year.head()

0    2020.0
1       NaN
2    2020.0
3    2020.0
4    2020.0
Name: SLA_Date, dtype: float64

In [4]:
import pandas as pd
Invoice_Combined_df = combined_Invoice
SLA_Combined_df = combined_sla

combined_Invoice.head()
SLA_Combined_df.head()

,Link_ID,SLA_Date,Last_Mile,Capacity_in_Mbps,Location,MRC_Excl,SLM_Comments,QRC_Incl,rank,SLA_ID,Unique_Link_Identifier_SLA,Service_Provider
0,1750,2020-10-01,Fibre,90,KRA - TIMES TOWERS TO KRA - KIXP (EADC),270000.0,"16/04/2022: Upgraded 90 Mbps @ KES 649,050 ->...",939600.00,0,2020-10-LTK00,2020-10-LTK00_1750,LTK
1,5277,2020-10-01,Fibre,75,KRA CUSTOMS HSE TO KRA TIMES TOWER,179718.0,Not in Invoice/Renamed/Relocated,625418.64,0,2020-10-LTK00,2020-10-LTK00_5277,LTK
2,5611,2020-10-01,Fibre,8,KRA AIRPORT CONTAINER DEPOT TO CUSTOMS HSE MSA,20642.5,Okay. No change,71835.90,0,2020-10-LTK00,2020-10-LTK00_5611,LTK
3,20103,2020-10-01,Fibre,9,KRA Wilson Airport Customs to Times Towers - V...,22950.0,Okay. No change,79866.00,0,2020-10-LTK00,2020-10-LTK00_20103,LTK
4,20954,2020-10-01,Fibre,20,KRA TIMES TOWERS NBO TO EGERTON UNIVERSITY NJO...,50000.0,Okay. No change,174000.00,0,2020-10-LTK00,2020-10-LTK00_20954,LTK


In [6]:
import pandas as pd
import numpy as np
Invoice_Combined_df = combined_Invoice
SLA_Combined_df = combined_sla


Merged_Data_SLA_Invoice = pd.DataFrame({
    'Modified_Link_ID': pd.concat([Invoice_Combined_df['modified_Link_ID'],SLA_Combined_df['Unique_Link_Identifier_SLA']]).unique()
})


#Inserting Service Provider values
SLA_SP = dict(zip(SLA_Combined_df['Unique_Link_Identifier_SLA'], SLA_Combined_df['Service_Provider']))
invoice_SP = dict(zip(Invoice_Combined_df['modified_Link_ID'], Invoice_Combined_df['Service_Provider']))

Merged_Data_SLA_Invoice['Service_Provider'] = Merged_Data_SLA_Invoice['Modified_Link_ID'].map(SLA_SP)
Merged_Data_SLA_Invoice['Service_Provider'] = np.where(
    Merged_Data_SLA_Invoice['Service_Provider'].isnull(),
    Merged_Data_SLA_Invoice['Modified_Link_ID'].map(invoice_SP),
    Merged_Data_SLA_Invoice['Service_Provider']
)
# SLA vs Invoice Status
Merged_Data_SLA_Invoice['In_SLA_vs_Invoiced'] = np.where(
    Merged_Data_SLA_Invoice['Modified_Link_ID'].isin(SLA_Combined_df['Unique_Link_Identifier_SLA']) &
    Merged_Data_SLA_Invoice['Modified_Link_ID'].isin(Invoice_Combined_df['modified_Link_ID']),
    "In SLA and Invoiced",
    np.where(Merged_Data_SLA_Invoice['Modified_Link_ID'].isin(SLA_Combined_df['Unique_Link_Identifier_SLA']),
             "IN SLA but Not Invoiced",
             np.where(
                 Merged_Data_SLA_Invoice['Modified_Link_ID'].isin(Invoice_Combined_df['modified_Link_ID']),
                 "Invoiced but not in SLA",
                 "Unknown"
             )
        )
    
)

In [ ]:
Merged_Data_SLA_Invoice

#### Adding SLA_QRC and Invoice QRC Value


In [1]:
import pandas as pd
ltk_sla_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\LTK_SLA.csv')
saf_sla_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\Saf_SLA.csv')
combined_sla = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\SLA_Combined.csv')

ltk_invoice_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\LTK_Invoice.csv')
saf_invoice_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\Saf_Invoice.csv')
combined_Invoice = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\Invoice_Combined.csv')

In [8]:
combined_sla.head()


,Link_ID,SLA_Date,Last_Mile,Capacity_in_Mbps,Location,MRC_Excl,SLM_Comments,QRC_Incl,SLA_ID,Unique_Link_Identifier_SLA,Service_Provider
0,1750,2020-10-01,Fibre,90,KRA KIXP,270000.0,"16/04/2022: Upgraded 90 Mbps @ KES 649,050 ->...",939600.00,2020-10-LTK00,2020-10-LTK00_1750,LTK
1,5277,2020-10-01,Fibre,75,KRA Customs Hse - Times Tower,179718.0,Not in Invoice/Renamed/Relocated,625418.64,2020-10-LTK00,2020-10-LTK00_5277,LTK
2,5611,2020-10-01,Fibre,8,KRA AIRPORT CONTAINER DEPOT TO CUSTOMS HSE MSA,20642.5,Okay. No change,71835.90,2020-10-LTK00,2020-10-LTK00_5611,LTK
3,20103,2020-10-01,Fibre,9,KRA Wilson Airport,22950.0,Okay. No change,79866.00,2020-10-LTK00,2020-10-LTK00_20103,LTK
4,20954,2020-10-01,Fibre,20,KRA Egerton University Njoro,50000.0,Okay. No change,174000.00,2020-10-LTK00,2020-10-LTK00_20954,LTK


In [20]:
invoice_aggr = combined_Invoice.groupby('modified_Link_ID').agg({'Total_QRC': 'max'}).reset_index()
df_merged = combined_Invoice.merge(invoice_aggr, on = 'modified_Link_ID', how='left')

In [22]:
df_merged.head()

,Invoice_Date,Link_ID,SLA_Date,Invoice_Period,Invoice_Description,Invoice_Reference,Total_QRC_x,modified_Link_ID,Unique_Link_Identifier_Invoice,Service_Provider,Total_QRC_y
0,2023-10-13,26046,2020-10-01,01-Oct-2023 to 31-Dec-2023,26046 National MPLS KRA MBITA TO VRF (INS-57),CRN-15473,-55986.24,2020-10-LTK00_26046,CRN-15473_26046_2020,LTK,55986.24
1,2023-10-06,C-00159-0087,NaN,01-Oct-2023 to 31-Dec-2023,C-00159-0087 EPL Connection charge 80MBPS->SAM...,116426,417600.00,nan-nan-LTK01_C-00159-0087,116426_C-00159-0087_0,LTK,417600.00
2,2023-10-06,1750,2020-10-01,01-Oct-2023 to 31-Dec-2023,01750 National MPLS KRA - TIMES TOWERS TO VRF,116426,939600.00,2020-10-LTK00_1750,116426_1750_2020,LTK,939600.00
3,2023-10-06,5611,2020-10-01,01-Oct-2023 to 31-Dec-2023,05611 National MPLS Msa link vlan 460 customs,116426,71835.90,2020-10-LTK00_5611,116426_5611_2020,LTK,71835.90
4,2023-10-06,20103,2020-10-01,01-Oct-2023 to 31-Dec-2023,20103 Leased_Circuits_Naticnal KRA WILSON AIRP...,116426,79866.00,2020-10-LTK00_20103,116426_20103_2020,LTK,79866.00


In [18]:
combined_sla.head()

,Link_ID,SLA_Date,Last_Mile,Capacity_in_Mbps,Location,MRC_Excl,SLM_Comments,QRC_Incl,SLA_ID,Unique_Link_Identifier_SLA,Service_Provider,Invoice_QRC
0,1750,2020-10-01,Fibre,90,KRA - TIMES TOWERS TO KRA - KIXP (EADC),270000.0,"16/04/2022: Upgraded 90 Mbps @ KES 649,050 ->...",939600.00,2020-10-LTK00,2020-10-LTK00_1750,LTK,55986.24
1,5277,2020-10-01,Fibre,75,KRA CUSTOMS HSE TO KRA TIMES TOWER,179718.0,Not in Invoice/Renamed/Relocated,625418.64,2020-10-LTK00,2020-10-LTK00_5277,LTK,417600.00
2,5611,2020-10-01,Fibre,8,KRA AIRPORT CONTAINER DEPOT TO CUSTOMS HSE MSA,20642.5,Okay. No change,71835.90,2020-10-LTK00,2020-10-LTK00_5611,LTK,939600.00
3,20103,2020-10-01,Fibre,9,KRA Wilson Airport Customs to Times Towers - V...,22950.0,Okay. No change,79866.00,2020-10-LTK00,2020-10-LTK00_20103,LTK,71835.90
4,20954,2020-10-01,Fibre,20,KRA TIMES TOWERS NBO TO EGERTON UNIVERSITY NJO...,50000.0,Okay. No change,174000.00,2020-10-LTK00,2020-10-LTK00_20954,LTK,79866.00


In [19]:
import pandas as pd

# Sample data
df1 = pd.DataFrame({'key': ['A', 'B', 'C', 'D'],
                    'value': [1, 2, 3, 4]})
df2 = pd.DataFrame({'key': ['B', 'B', 'D', 'F'],
                    'value2': [10, 20, 30, 40]})

# Group and aggregate df2 to get max value
grouped_df2 = df2.groupby('key').agg({'value2': 'max'}).reset_index()

# Merge based on 'key' column
merged_df = pd.merge(df1, grouped_df2, on='key', how='left')

print(merged_df)

  key  value  value2
0   A      1     NaN
1   B      2    20.0
2   C      3     NaN
3   D      4    30.0


In [13]:
import math
year = pd.to_datetime(combined_sla['SLA_Date']).dt.year.fillna(0).astype(int).astype(str).replace('0','0000')
year

0     2020
1     2020
2     2020
3     2020
4     2020
      ... 
80    2023
81    2023
82    2023
83    2023
84    2023
Name: SLA_Date, Length: 85, dtype: object

In [10]:
import pandas as pd

# Sample data
data = {'SLA_Date': ['2022-01-15', '2023-02-20', pd.NA, '2024-04-10']}
df = pd.DataFrame(data)

# Convert to datetime, extract year, convert to string, fillna with '0000'
df['Year'] = pd.to_datetime(df['SLA_Date']).dt.year.fillna(0).astype(int).astype(str).replace('0','0000')

print(df)


     SLA_Date  Year
0  2022-01-15  2022
1  2023-02-20  2023
2        <NA>  0000
3  2024-04-10  2024


In [14]:
x = 5

In [44]:
def Invoice_p_pro(prd):
    prd_col = pd.to_datetime(prd.split(' ')[0])
    year = prd_col.year
    month = prd_col.month

    if month >= 1 and month <= 3:
        quarter ='Q3'
    elif month >= 4 & month <= 6:
        quarter = 'Q4'
    elif month >= 7 & month <= 9:
        quarter = 'Q1'
    elif month >= 10 & month <= 12:
        quarter = 'Q2'
    else:
        quarter = 'Unknown'
    
    #Fyr
    if month >= 1 & month <= 6:
        fyr = f'Fyr {year-1}/{str(year)[-2:]}'
    elif month >= 7 & month <= 12:
        fyr = f'Fyr {year}/{str(year+1)[-2:]}'
    else:
        fyr = 'unkown'

    return quarter,fyr

x,y = combined_Invoice['Invoice_Period'].apply(Invoice_p_pro).apply(pd.Series)

In [47]:
y

1

In [32]:
pr_col = pd.to_datetime(combined_Invoice['Invoice_Period'].str.split(' ', expand=True)[0])
year = pr_col.dt.year
month = pr_col.dt.month
if month >= 1 and month <= 3:
    quarter ='Q3'
elif month >= 4 & month <= 6:
    quarter = 'Q4'
elif month >= 7 & month <= 9:
    quarter = 'Q1'
elif month >= 10 & month <= 12:
    quarter = 'Q2'
else:
    quarter = 'Unknown'

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [33]:
if month >= 1 and month <= 3:
    quarter = 'Q3'
elif month >= 4 and month <= 6:
    quarter = 'Q4'
elif month >= 7 and month <= 9:
    quarter = 'Q1'
elif month >= 10 and month <= 12:
    quarter = 'Q2'
else:
    quarter = 'Unknown'


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [49]:
import pandas as pd

def Invoice_p_pro(prd):
    prd_col = pd.to_datetime(prd.split(' ')[0])
    year = prd_col.year
    month = prd_col.month

    if month >= 1 and month <= 3:
        quarter = 'Q3'
    elif month >= 4 and month <= 6:
        quarter = 'Q4'
    elif month >= 7 and month <= 9:
        quarter = 'Q1'
    elif month >= 10 and month <= 12:
        quarter = 'Q2'
    else:
        quarter = 'Unknown'
    
    # Financial Year
    if month >= 1 and month <= 6:
        fyr = f'Fyr {year-1}/{str(year)[-2:]}'
    elif month >= 7 and month <= 12:
        fyr = f'Fyr {year}/{str(year+1)[-2:]}'
    else:
        fyr = 'Unknown'

    return quarter, fyr

# Applying the function to the DataFrame column
combined_Invoice[['Quarter', 'Financial_Year']] = combined_Invoice['Invoice_Period'].apply(Invoice_p_pro).apply(pd.Series)

# Check the resulting DataFrame
print(combined_Invoice[['Invoice_Period', 'Quarter', 'Financial_Year']].head())


combined_Invoice

               Invoice_Period Quarter Financial_Year
0  01-Oct-2023 to 31-Dec-2023      Q2    Fyr 2023/24
1  01-Oct-2023 to 31-Dec-2023      Q2    Fyr 2023/24
2  01-Oct-2023 to 31-Dec-2023      Q2    Fyr 2023/24
3  01-Oct-2023 to 31-Dec-2023      Q2    Fyr 2023/24
4  01-Oct-2023 to 31-Dec-2023      Q2    Fyr 2023/24


,Invoice_Date,Link_ID,SLA_Date,Invoice_Period,Invoice_Description,Invoice_Reference,Total_QRC,modified_Link_ID,Unique_Link_Identifier_Invoice,Service_Provider,Quarter,Financial_Year
0,2023-10-13,26046,2020-10-01,01-Oct-2023 to 31-Dec-2023,26046 National MPLS KRA MBITA TO VRF (INS-57),CRN-15473,-55986.2400,2020-10-LTK00_26046,CRN-15473_26046_2020,LTK,Q2,Fyr 2023/24
1,2023-10-06,C-00159-0087,NaN,01-Oct-2023 to 31-Dec-2023,C-00159-0087 EPL Connection charge 80MBPS->SAM...,116426,417600.0000,nan-nan-LTK01_C-00159-0087,116426_C-00159-0087_0,LTK,Q2,Fyr 2023/24
2,2023-10-06,1750,2020-10-01,01-Oct-2023 to 31-Dec-2023,01750 National MPLS KRA - TIMES TOWERS TO VRF,116426,939600.0000,2020-10-LTK00_1750,116426_1750_2020,LTK,Q2,Fyr 2023/24
3,2023-10-06,5611,2020-10-01,01-Oct-2023 to 31-Dec-2023,05611 National MPLS Msa link vlan 460 customs,116426,71835.9000,2020-10-LTK00_5611,116426_5611_2020,LTK,Q2,Fyr 2023/24
4,2023-10-06,20103,2020-10-01,01-Oct-2023 to 31-Dec-2023,20103 Leased_Circuits_Naticnal KRA WILSON AIRP...,116426,79866.0000,2020-10-LTK00_20103,116426_20103_2020,LTK,Q2,Fyr 2023/24
...,...,...,...,...,...,...,...,...,...,...,...,...
80,2023-10-01,95096538,2020-12-16,01-Oct-2023 to 31-Dec-2023,KRA BUSIA-WIMAX,B1-10096515502,280173.2166,2020-12-Saf00_95096538,B1-10096515502_95096538_2020,Saf,Q2,Fyr 2023/24
81,2023-10-01,95096538,2023-10-01,01-Oct-2023 to 31-Dec-2023,KRA BUSIA-WIMAX,B1-10096515502,280173.2166,2023-10-Saf01_95096538,B1-10096515502_95096538_2023,Saf,Q2,Fyr 2023/24
82,2023-10-01,95096687,2020-12-16,01-Oct-2023 to 31-Dec-2023,KRA CUSTOMS OFFICE- NAIROBI-NAMANGA WIMAX,B1-10096515502,280129.9950,2020-12-Saf00_95096687,B1-10096515502_95096687_2020,Saf,Q2,Fyr 2023/24
83,2023-10-01,95096716,2020-12-16,01-Oct-2023 to 31-Dec-2023,KRA ISEBANIA-WIMAX,B1-10096515502,294887.7702,2020-12-Saf00_95096716,B1-10096515502_95096716_2020,Saf,Q2,Fyr 2023/24


##### Locations Processing 

In [2]:
import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt


In [17]:
locs = pd.read_json('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\SLA_ETL\\KRA Stations.json')

In [30]:
keys = list(locs['markers'][0].keys())  # Column Names 
location_df = {}
for col,val in locs['markers'].items():
    if col in location_df.keys():
        location_df[col].append(val)
    else:
        location_df[col] = val


In [37]:
location_df = {}
for row in locs['markers']:
    for col, val in row.items():
        if col in location_df.keys():
            location_df[col].append(val)
        else:
            location_df[col] = val

AttributeError: 'str' object has no attribute 'append'

In [31]:
location_df

{0: {'latitude': '-1.290278',
  'longitude': '36.8217003',
  'shopaddress': 'Head Quarters\r\nBox. 48240 - 00100.\r\nTel. +254 20 281 0000',
  'shopemailaddress': 'mauriceterer@gmail.com',
  'shopname': 'Nairobi - Times Tower Building',
  'shopphonenumber': '+254 20 281 0000',
  'type': 'office',
  'weekdayclosing': 'Friday',
  'weekdayopening': 'Monday 8:00am - 5:00pm',
  'weekendclosing': '',
  'weekendopening': ''},
 1: {'latitude': '-0.285359',
  'longitude': '36.064905',
  'shopaddress': 'Domestic Taxes Dpmt.\r\nBox. 270- 20100.\r\nTel. 051-2213926, 0725-777409',
  'shopemailaddress': '',
  'shopname': 'Nakuru - Generation House',
  'shopphonenumber': '',
  'type': 'office',
  'weekdayclosing': 'Friday',
  'weekdayopening': 'Monday 8:00 am - 5:00 pm',
  'weekendclosing': '',
  'weekendopening': ''},
 2: {'latitude': '-1.3298749',
  'longitude': '36.865326',
  'shopaddress': 'P.O. Box 46285-00100 Nairobi\r\n',
  'shopemailaddress': '',
  'shopname': 'Nairobi - Sameer Park',
  'shop

In [ ]:
locs['markers'].items()

In [24]:


loc_df = pd.DataFrame(locs['markers'])

loc_df.head()

,markers
0,"{'latitude': '-1.290278', 'longitude': '36.821..."
1,"{'latitude': '-0.285359', 'longitude': '36.064..."
2,"{'latitude': '-1.3298749', 'longitude': '36.86..."
3,"{'latitude': '-1.258983072', 'longitude': '36...."
4,"{'latitude': '0.521120', 'longitude': '35.2682..."


In [26]:
locs['markers']

0     {'latitude': '-1.290278', 'longitude': '36.821...
1     {'latitude': '-0.285359', 'longitude': '36.064...
2     {'latitude': '-1.3298749', 'longitude': '36.86...
3     {'latitude': '-1.258983072', 'longitude': '36....
4     {'latitude': '0.521120', 'longitude': '35.2682...
5     {'latitude': '3.124411', 'longitude': '35.6116...
6     {'latitude': '1.01553', 'longitude': '35.00110...
7     {'latitude': '-1.5162387', 'longitude': '37.26...
8     {'latitude': '-0.2851198', 'longitude': '36.06...
9     {'latitude': '-1.0339247', 'longitude': '37.07...
10    {'latitude': '-0.4277859', 'longitude': '36.95...
11    {'latitude': '-0.503365', 'longitude': '37.274...
12    {'latitude': '0.0071471', 'longitude': '37.071...
13    {'latitude': '-0.752162', 'longitude': '37.064...
14    {'latitude': '1.1651', 'longitude': '36.8241',...
15    {'latitude': '-4.0633338', 'longitude': '39.66...
16    {'latitude': '-3.206917', 'longitude': '40.116...
17    {'latitude': '-3.393726', 'longitude': '38

In [15]:
keys

['latitude',
 'longitude',
 'shopaddress',
 'shopemailaddress',
 'shopname',
 'shopphonenumber',
 'type',
 'weekdayclosing',
 'weekdayopening',
 'weekendclosing',
 'weekendopening']

In [46]:
# Creating Empty Cols
location_dict = {col: [] for col in list(locs['markers'][0].keys())}
for row

In [73]:
locs = pd.read_json('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\SLA_ETL\\KRA Stations.json')  #Loading the File
location_dict = {col: [] for col in list(locs['markers'][0].keys())}  #Creating the dict with keys but empty values
for row in locs['markers']:
    for col, val in row.items():
        location_dict[col].append(val)

location_df = pd.DataFrame(location_dict)  #Converting the dictinary into a pd.dataframe

wanted_cols = ['latitude', 'longitude', 'shopname']
kra_location_df = location_df.copy()[wanted_cols] #Subsetting long, lat and name of the TSO
kra_location_df['latitude'] = kra_location_df['latitude'].astype(float) 
kra_location_df['longitude'] = kra_location_df['longitude'].astype(float)



In [15]:
import pandas as pd

# Load the JSON file into a DataFrame
locs = pd.read_json('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\SLA_ETL\\KRA Stations.json')

# Extract relevant columns and convert data types
wanted_cols = ['latitude', 'longitude', 'shopname']
kra_location_df = locs['markers'].apply(pd.Series)[wanted_cols]


kra_location_df['latitude'] = kra_location_df['latitude'].astype(float)
kra_location_df['longitude'] = kra_location_df['longitude'].astype(float)



# Now kra_location_df contains only the desired columns with correct data types
kra_location_df.head()
kra_location_df.rename(columns = {'shopname': 'TSO Name', 'latitude': 'lat', 'longitude': 'lon'}, inplace = True)

In [27]:
kra_location_df.head()
kra_location_df['TSO Name'].value_counts()

TSO Name
Nairobi - Sameer Park                                 2
Nakuru - Generation House                             2
Nairobi - Times Tower Building                        1
Mombasa - Customs House                               1
Busia OSBP                                            1
Malaba OSBP                                           1
Kisii - Umoja Plaza                                   1
Kakamega - Posta House                                1
Kisumu - Lake Basin Mall                              1
Ukunda - Holding Meridian                             1
Lamu - Mkunguni Square                                1
Voi - Potter's House                                  1
Malindi - Malindi Complex                             1
Murang'a - Ngeka Centre                               1
Kiambu Mall                                           1
Nanyuki - Ubii Plaza                                  1
Kerugoya - Machere Plaza                              1
KRA Central Region Headquarters - Nyeri

In [8]:
combined_sla['Location'].replace('KRA', '', inplace= True)

,lat,lon,TSO Name
0,-1.290278,36.821700,Nairobi - Times Tower Building
1,-0.285359,36.064905,Nakuru - Generation House
2,-1.329875,36.865326,Nairobi - Sameer Park
3,-1.258983,36.795028,Nairobi - Ushuru Pension Towers
4,0.521120,35.268203,Eldoret
5,3.124411,35.611669,Lodwar - Customs House
6,1.015530,35.001108,Kitale - Trans - Nzoia Women Plaza
7,-1.516239,37.261634,"Machakos, Sartaj Building, 3rd Floor, off Kang..."
8,-0.285120,36.063309,Nakuru - Generation House
9,-1.033925,37.073021,Thika - Thika House


In [36]:
combined_sla['Location'] = combined_sla['Location'].replace({'KRA': '',
                                                                 '-': ''}, regex=True).str.strip()
combined_sla

,Link_ID,SLA_Date,Last_Mile,Capacity_in_Mbps,Location,MRC_Excl,SLM_Comments,QRC_Incl,SLA_ID,Unique_Link_Identifier_SLA,Service_Provider
0,1750,2020-10-01,Fibre,90,KIXP,270000.0,"16/04/2022: Upgraded 90 Mbps @ KES 649,050 ->...",939600.00,2020-10-LTK00,2020-10-LTK00_1750,LTK
1,5277,2020-10-01,Fibre,75,Customs Hse Times Tower,179718.0,Not in Invoice/Renamed/Relocated,625418.64,2020-10-LTK00,2020-10-LTK00_5277,LTK
2,5611,2020-10-01,Fibre,8,AIRPORT CONTAINER DEPOT TO CUSTOMS HSE MSA,20642.5,Okay. No change,71835.90,2020-10-LTK00,2020-10-LTK00_5611,LTK
3,20103,2020-10-01,Fibre,9,Wilson Airport,22950.0,Okay. No change,79866.00,2020-10-LTK00,2020-10-LTK00_20103,LTK
4,20954,2020-10-01,Fibre,20,Egerton University Njoro,50000.0,Okay. No change,174000.00,2020-10-LTK00,2020-10-LTK00_20954,LTK
...,...,...,...,...,...,...,...,...,...,...,...
80,95096538,2023-10-01,Unknown,20,BUSIAFIBRE,NaN,NaN,NaN,2023-10-Saf01,2023-10-Saf01_95096538,Saf
81,95006961,2023-10-01,Unknown,15,FIXED 30th Floor 3001,NaN,NaN,NaN,2023-10-Saf01,2023-10-Saf01_95006961,Saf
82,95006963,2023-10-01,Unknown,15,FIXED 25th Floor 2575,NaN,NaN,NaN,2023-10-Saf01,2023-10-Saf01_95006963,Saf
83,95021939,2023-10-01,Unknown,15,KENYA SCHOOL OF REVENUE FIBER,NaN,NaN,NaN,2023-10-Saf01,2023-10-Saf01_95021939,Saf


In [12]:
location_df.head()
location_df.info()

NameError: name 'location_df' is not defined

In [7]:
import plotly.express as px

# Create a scatter map using Plotly Express
fig = px.scatter_mapbox(combined_sla, lat='lat', lon='lon', hover_name='TSO Name',
                        zoom=5, height=500)

# Customize map layout and style
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(title='Geographical Distribution of Shops')
fig.show()

In [10]:
combined_sla['Location'].unique()

array(['KRA KIXP', 'KRA Customs Hse - Times Tower',
       'KRA AIRPORT CONTAINER DEPOT TO CUSTOMS HSE MSA',
       'KRA Wilson Airport', 'KRA Egerton University Njoro',
       'KRA Sameer Park', 'KRA -ICD Naivasha SGR', 'KRA City Hall',
       'KRA Kajiado', 'KRA Narok', 'KRA Nairobi PPO',
       'KRA JKIA Terminal 2', 'KRA Kisumu Pier', 'KRA Kitale',
       'KRA Lamu Airport', 'KRA Kisumu Post Parcels (PPO)',
       'KRA Kisumu Lake Basin Mall', 'KRA Kisumu Airport', 'KRA Maralal',
       'KRA MIA Mombasa Baggage Hall', 'KRA Lodwar', 'CBC Upper Hill',
       'WAN Backhaul', 'KRA Sameer Park MRI', 'JKIA Signon',
       'JKIA Exports', 'BUNGOMA MICROWAVE', 'BUSIA-WIMAX',
       'DR BACKHAUL FIBRE', 'ELDORET KIPTAGICH MPLS FIBER',
       'ELDORET KPC LOOP MICROWAVE', 'HQ-INTERNET',
       'FORODHA JKIA LOOP FIBER', 'FORTIS-WESTLANDS LOOP FIBER',
       'GARISSA LOOP MICROWAVE', 'ISIOLO LOOP MICROWAVE',
       'ISEBANIA-WIMAX', 'JKIA ACHIL SHED LOOP FIBER', 'JKIA SWISSPORT',
       'KAKA

In [11]:
import urllib.request
url = 'https://open.africa/api/3/action/datastore_search?resource_id=25d2357f-d0b2-4ad9-8e8f-aa95cc149e42&limit=5&q=title:jones'  
fileobj = urllib.request.urlopen(url)
print(fileobj.read())
https://open.africa/dataset/sensorsafrica-airquality-archive-nairobi

b'{"help": "https://open.africa/api/3/action/help_show?name=datastore_search", "success": true, "result": {"include_total": true, "limit": 5, "q": "title:jones", "records_format": "objects", "resource_id": "25d2357f-d0b2-4ad9-8e8f-aa95cc149e42", "total_estimation_threshold": null, "records": [], "fields": [{"id": "_id", "type": "int"}, {"id": "sensor_id", "type": "numeric"}, {"id": "sensor_type", "type": "text"}, {"id": "location", "type": "numeric"}, {"id": "lat", "type": "numeric"}, {"id": "lon", "type": "numeric"}, {"id": "timestamp", "type": "timestamp"}, {"id": "value_type", "type": "text"}, {"id": "value", "type": "numeric"}, {"id": "rank", "type": "float"}], "_links": {"start": "/api/3/action/datastore_search?resource_id=25d2357f-d0b2-4ad9-8e8f-aa95cc149e42&limit=5&q=title%3Ajones", "next": "/api/3/action/datastore_search?resource_id=25d2357f-d0b2-4ad9-8e8f-aa95cc149e42&limit=5&q=title%3Ajones&offset=5"}, "total": 0, "total_was_estimated": false}}'


In [14]:
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
air_quality = fetch_ucirepo(id=360) 
  
# data (as pandas dataframes) 
X = air_quality.data.features 
y = air_quality.data.targets 
  
# metadata 
print(air_quality.metadata) 
  
# variable information 
print(air_quality.variables) 


{'uci_id': 360, 'name': 'Air Quality', 'repository_url': 'https://archive.ics.uci.edu/dataset/360/air+quality', 'data_url': 'https://archive.ics.uci.edu/static/public/360/data.csv', 'abstract': 'Contains the responses of a gas multisensor device deployed on the field in an Italian city. Hourly responses averages are recorded along with gas concentrations references from a certified analyzer. ', 'area': 'Computer Science', 'tasks': ['Regression'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 9358, 'num_features': 15, 'feature_types': ['Real'], 'demographics': [], 'target_col': None, 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Sun Mar 10 2024', 'dataset_doi': '10.24432/C59K5F', 'creators': ['Saverio Vito'], 'intro_paper': {'title': 'On field calibration of an electronic nose for benzene estimation in an urban pollution monitoring scenario', 'authors': 'S. D. Vito, E. Massera, M. P

In [15]:
print(air_quality.metadata) 

{'uci_id': 360, 'name': 'Air Quality', 'repository_url': 'https://archive.ics.uci.edu/dataset/360/air+quality', 'data_url': 'https://archive.ics.uci.edu/static/public/360/data.csv', 'abstract': 'Contains the responses of a gas multisensor device deployed on the field in an Italian city. Hourly responses averages are recorded along with gas concentrations references from a certified analyzer. ', 'area': 'Computer Science', 'tasks': ['Regression'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 9358, 'num_features': 15, 'feature_types': ['Real'], 'demographics': [], 'target_col': None, 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Sun Mar 10 2024', 'dataset_doi': '10.24432/C59K5F', 'creators': ['Saverio Vito'], 'intro_paper': {'title': 'On field calibration of an electronic nose for benzene estimation in an urban pollution monitoring scenario', 'authors': 'S. D. Vito, E. Massera, M. P

In [53]:
import plotly.graph_objects as go
import pandas as pd

# Example DataFrame

# Create map
fig = go.Figure()

# Add scatter mapbox layer with markers and text
fig.add_trace(go.Scattermapbox(
    lat=combined_sla['lat'],
    lon=combined_sla['lon'],
    mode='markers+text',  # Ensures markers and text are shown
    marker=go.scattermapbox.Marker(size=9),
    text=combined_sla['Location'],  # Text labels to be shown
    textposition='top right',  # Adjust text position as needed
    hoverinfo='text',
    hovertext=combined_sla['Location']  # This will show the TSO Name on hover
))

# Customize map layout
fig.update_layout(
    mapbox=dict(
        style='open-street-map',
        zoom=5,
        center=dict(lat=0, lon=37)
    ),
    title='Geographical Distribution of Shops',
    height=500
)

# Show map
fig.show()


##### Combining the Cordinates and TSO Name 

In [2]:
import pandas as pd
ltk_sla_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\LTK_SLA.csv')
saf_sla_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\Saf_SLA.csv')
combined_sla = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\SLA_Combined.csv')

ltk_invoice_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\LTK_Invoice.csv')
saf_invoice_df = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\Saf_Invoice.csv')
combined_Invoice = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\Invoice_Combined.csv')

In [57]:
import pandas as pd

# Load the JSON file into a DataFrame
locs = pd.read_json('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\SLA_ETL\\KRA Stations.json')

# Extract relevant columns and convert data types
wanted_cols = ['latitude', 'longitude', 'shopname']
kra_location_df = locs['markers'].apply(pd.Series)[wanted_cols]


kra_location_df['latitude'] = kra_location_df['latitude'].astype(float)
kra_location_df['longitude'] = kra_location_df['longitude'].astype(float)



# Now kra_location_df contains only the desired columns with correct data types
kra_location_df.head()
kra_location_df.rename(columns = {'shopname': 'TSO Name', 'latitude': 'lat', 'longitude': 'lon'}, inplace = True)

In [62]:
# adding Location not found in table table
kra_location_df.drop_duplicates()


#Creating the Location Table and adding the TSO in Our Dataset
locations_cor_df = pd.DataFrame(
    {
        'TSO_name' : combined_sla['Location'].replace({'KRA': '',
                                                        '-': '',
                                                        'INTERNET': ''}, regex=True).str.strip().str.title()
    }
)


def find_coordinates(tso_name, kra_location_df):
    for _, row in kra_location_df.iterrows():
        if row['TSO Name'].split('-')[0].strip().lower() in tso_name.lower():
            return row['TSO Name']
        elif row['TSO Name'].split(' ')[0].strip().lower() in tso_name.lower():
            return row['TSO Name']
        elif row['TSO Name'].split(',')[0].strip().lower() in tso_name.lower():
            return row['TSO Name']
    if tso_name.split(' ')[0].lower() in ['fixed', 'city', 'hq', 'kixp']:
        return 'Nairobi - Times Tower Building'    
    elif tso_name.split(' ')[0].lower() in ['jkia', 'forodha']:
        return 'JKIA'
    return 'None'

# Apply the function to the 'TSO_name' column to create 'Coordinates' column
locations_cor_df['Search Key'] = locations_cor_df['TSO_name'].apply(lambda x: find_coordinates(x, kra_location_df))


locations_cor_df


,TSO_name,Search Key
0,Kixp,None
1,Customs Hse Times Tower,None
2,Airport Container Depot To Customs Hse Msa,None
3,Wilson Airport,None
4,Egerton University Njoro,None
...,...,...
80,Busiafibre,Busia OSBP
81,Fixed 30Th Floor 3001,Nairobi - Times Tower Building
82,Fixed 25Th Floor 2575,Nairobi - Times Tower Building
83,Kenya School Of Revenue Fiber,None


In [48]:
import pandas as pd

# Sample data for illustration
combined_sla = pd.DataFrame({
    'Location': ['KRA-ABC-123', 'KRA-DEF-456', 'KRA-GHI-789']
})

kra_location_df = pd.DataFrame({
    'TSO Name': ['ABC', 'DEF', 'GHI'],
    'Coordinates': ['123,456', '789,012', '345,678']  # Example coordinates
})

# Creating the Location Table and adding the TSO in Our Dataset
locations_cor_df = pd.DataFrame({
    'TSO_name': combined_sla['Location'].replace({'KRA': '', '-': ' '}, regex=True).str.strip().str.title()
})

# Function to find the matching TSO Name and return the corresponding coordinates
def find_coordinates(tso_name, kra_location_df):
    for _, row in kra_location_df.iterrows():
        if row['TSO Name'] in tso_name:
            return row['Coordinates']
    return 'None'

# Apply the function to the 'TSO_name' column to create 'Coordinates' column
locations_cor_df['Coordinates'] = locations_cor_df['TSO_name'].apply(lambda x: find_coordinates(x, kra_location_df))

print(locations_cor_df)


  TSO_name Coordinates
0  Abc 123        None
1  Def 456        None
2  Ghi 789        None


##### Nominatim API Approach

In [162]:
!pip install  python-Levenshtein
#import fuzz

   ---------------------------------------- 0.0/98.3 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/98.3 kB ? eta -:--:--
   ------------ --------------------------- 30.7/98.3 kB 435.7 kB/s eta 0:00:01
   ------------ --------------------------- 30.7/98.3 kB 435.7 kB/s eta 0:00:01
   ------------------------ --------------- 61.4/98.3 kB 297.7 kB/s eta 0:00:01
   ---------------------------------------- 98.3/98.3 kB 431.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.7 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.7 MB 6.4 MB/s eta 0:00:01
   -- ------------------------------------- 0.1/1.7 MB 6.4 MB/s eta 0:00:01
   --- ------------------------------------ 0.1/1.7 MB 944.1 kB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.7 MB 958.4 kB/s eta 0:00:02
   ------ --------------------------------- 0.3/1.7 MB 1.0 MB/s eta 0:00:02
   ------ -------

In [8]:
import requests
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

def get_coordinates(location_name, depth=0):
    base_url = 'https://nominatim.openstreetmap.org/search'
    params = {'q': location_name, 'format': 'json', 'limit': 5}
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(base_url, params=params, headers=headers)
    data = response.json()

    if data:
        best_match = None
        max_ratio = 0
        for result in data:
            if any(country in result.get('display_name', '') for country in ['Kenya', 'Uganda', 'Tanzania', 'Rwanda', 'Burundi']):
                ratio = fuzz.partial_ratio(location_name.lower(), result.get('display_name', '').lower())
                if ratio > max_ratio:
                    max_ratio = ratio
                    best_match = result
        
        if best_match:
            latitude = best_match['lat']
            longitude = best_match['lon']
            return latitude, longitude

    if depth == 0:
        parts = location_name.split()
        if len(parts) > 1:
            first_part = parts[0]
            second_part = parts[1]
            if first_part != location_name:
                first_part_result = get_coordinates(first_part, depth + 1)
                if first_part_result != (None, None):
                    return first_part_result

            if second_part != location_name:
                second_part_result = get_coordinates(second_part, depth + 1)
                if second_part_result != (None, None):
                    return second_part_result

    return None, None



#applying the function into our data
#combined_sla[['lat', 'lon']] = combined_sla['Location'].apply(lambda x: pd.Series(get_coordinates(x)))

In [22]:
#Cleaning up the locations col
combined_sla = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\SLA_Combined.csv')

#defining a function to clean the location col
def clean_location(location):
    #Defining Partial Replacements
    partial_replacements = {
        'kra': '',
        '-': '',
        'internet': '',
        'ceragon': '',
        'wimax': '',
        'fiber': '',
        'loop': '',
        'microwave':'',
        'ppo': '',
        'pier': '',
        'fibre': ''

    }

    #Defining replacements for entire cellls
    full_replacement = {
        'sameer': 'Nairobi-Sameer Park',
        'hq' : 'Nairobi-Times Tower',
        'fixed' : 'Nairobi-Times Tower',
        'kixp' : 'Nairobi-Times Tower',
        'backhaul': 'Nairobi-Times Tower',
        'times tower' : 'Nairobi-Times Tower',
        'msa': 'Mombasa - Customs House',
        'city hall': 'Nairobi - City Hall',
        'kenya school': 'KESRA Nairobi',
        'kesra':'KESRA Nairobi',
        'fortiswestlands': 'Fortis westlands',
        'kopanga': 'kilindini'
        

    } 

    #Performing the partial replacements
    for key,value in partial_replacements.items():
        location = location.lower().replace(key, value)

    #Perform full replacements
    for key, value in full_replacement.items():
        if key in location.lower():
            return value
        
    # strip and title case the final string
    location = location.strip().title()

    return location

combined_sla['Location'] = combined_sla['Location'].apply(clean_location)

In [52]:
combined_sla = pd.read_csv('C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\SLA EDA\\SLA_ETL\\Processsed_Data\\SLA_Combined.csv')

In [24]:
#applying the function into our data
combined_sla[['lat', 'lon']] = combined_sla['Location'].apply(lambda x: pd.Series(get_coordinates(x))).astype(float)

In [25]:
combined_sla.head()

,Link_ID,SLA_Date,Last_Mile,Capacity_in_Mbps,Location,MRC_Excl,SLM_Comments,QRC_Incl,SLA_ID,Unique_Link_Identifier_SLA,Service_Provider,lat,lon
0,1750,2020-10-01,Fibre,90,Nairobi-Times Tower,270000.0,"16/04/2022: Upgraded 90 Mbps @ KES 649,050 ->...",939600.00,2020-10-LTK00,2020-10-LTK00_1750,LTK,-1.290405,36.824225
1,5277,2020-10-01,Fibre,75,Nairobi-Times Tower,179718.0,Not in Invoice/Renamed/Relocated,625418.64,2020-10-LTK00,2020-10-LTK00_5277,LTK,-1.290405,36.824225
2,5611,2020-10-01,Fibre,8,Mombasa - Customs House,20642.5,Okay. No change,71835.90,2020-10-LTK00,2020-10-LTK00_5611,LTK,-4.050520,39.667169
3,20103,2020-10-01,Fibre,9,Wilson Airport,22950.0,Okay. No change,79866.00,2020-10-LTK00,2020-10-LTK00_20103,LTK,-1.324121,36.811250
4,20954,2020-10-01,Fibre,20,Egerton University Njoro,50000.0,Okay. No change,174000.00,2020-10-LTK00,2020-10-LTK00_20954,LTK,-0.368569,35.932296


In [6]:
combined_sla = combined_sla[combined_sla['lat'].isna()]
combined_sla.head()

,Link_ID,SLA_Date,Last_Mile,Capacity_in_Mbps,Location,MRC_Excl,SLM_Comments,QRC_Incl,SLA_ID,Unique_Link_Identifier_SLA,Service_Provider,lat,lon
21,SVC-000001477,2020-10-01,Internet,30,Cbc Upper Hill,66700.0,"Upgraded to 300 MBPS @ KES 1,080,000 EX VAT, b...",266933.4,2020-10-LTK00,2020-10-LTK00_SVC-000001477,LTK,NaN,NaN
44,95044903,2020-12-16,MPLS,5,Lwakhakha,22500.0,Former Kilindini Gate 9,78300.0,2020-12-Saf00,2020-12-Saf00_95044903,Saf,NaN,NaN
56,95096536,2020-12-16,MPLS,12,Malaba,69997.5,NaN,243591.3,2020-12-Saf00,2020-12-Saf00_95096536,Saf,NaN,NaN


In [27]:
combined_sla.info()



<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Link_ID                     0 non-null      object 
 1   SLA_Date                    0 non-null      object 
 2   Last_Mile                   0 non-null      object 
 3   Capacity_in_Mbps            0 non-null      int64  
 4   Location                    0 non-null      object 
 5   MRC_Excl                    0 non-null      float64
 6   SLM_Comments                0 non-null      object 
 7   QRC_Incl                    0 non-null      float64
 8   SLA_ID                      0 non-null      object 
 9   Unique_Link_Identifier_SLA  0 non-null      object 
 10  Service_Provider            0 non-null      object 
 11  lat                         0 non-null      float64
 12  lon                         0 non-null      float64
dtypes: float64(4), int64(1), object(8)
memory usage: 0.0

In [47]:
def get_coordinates(location_name):
    base_url = 'https://nominatim.openstreetmap.org/search'
    params = {'q': location_name, 'format': 'json', 'limit': 1}
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(base_url, params = params, headers = headers)
    data = response.json()

    if data:
        for result in data:
            # Check if the result is within East Africa (Kenya, Uganda, Tanzania, Rwanda, Burundi)
            if any(country in result.get('display_name', '') for country in ['Kenya', 'Uganda']):
                latitude = result['lat']
                longitude = result['lon']
                return latitude, longitude
    
    return None, None

loc = 'Nairobi-City Square'
get_coordinates(loc)


('-1.2859147', '36.8208075')

In [51]:
loc = 'Lwakhakha'
get_coordinates(loc)

('1.0728347', '34.1772262')

In [44]:
words = []
word = ''

while word !=  'end':
    word = input("Enter a word: ")
    words.append(word)
story = ' '.join(words[:-1])
print(story)

Hello W


In [48]:
story = story + ' end'

In [49]:
story

'Hello W end end'

In [50]:
story[:-3]

'Hello W end '